# Программа по составлению и улучшению расписания

## Предварительные настройки

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My Drive/CourseWork/4 course/Mari/TimeTablePJCT

Mounted at /content/drive/
/content/drive/My Drive/CourseWork/4 course/Mari/TimeTablePJCT


In [ ]:
import os.path

import json
import copy
import random

from collections import Counter
# С 9.01 - 24.03 = 54 дня (с пн-пт) (11-12 недель)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import collections.abc
def updateObject(d, u):
    for k, v in u.items():
        if isinstance(v, collections.abc.Mapping):
            d[k] = updateObject(d.get(k, {}), v)
        else:
            d[k] = v
    return d

## Код системы

In [ ]:
# Утилиты для класса ModuleTimeTable:
class Utils:
    # ******************** [ Показ ошибок в одном формате ] ********************
    def error(self, msg, reason =''):
        print("[!ERROR!]:", msg)
        
        if len(reason) > 0:
            print(" * Reason:", reason)

In [ ]:
# =============================== [ Университет ] ===============================
class University(Utils):
# __________ [ University ] __________
    def __init__(self, auditorium_db_file_path = ''):

        self.DB_AUDITORIUM = None

        if len(auditorium_db_file_path) > 0:
            with open("auditorium_db.json", "r", encoding="utf-8") as file:
                self.DB_AUDITORIUM = json.load(file)
        
            self.DB_CORPUSES = [ corpus for corpus in self.DB_AUDITORIUM ]
            
        # Свое "ручное" заполнение данных
        else:
            self.DB_AUDITORIUM = None

            self.DB_CORPUSES = {
                "Корпус на ул. Львовская, 1В": 1,
                "Корпус на ул. Родионова, 136": 2,
                "Корпус на ул. Б.Печерская, 25/12": 3,
                "Корпус на ул. Сормовское шоссе, 30": 4
            }

        self.DB_SUBJECT_TEACHER = {}

        self.DB_STUDENTS_AMOUNT = {
            "17БИ-1": 30,
            "17БИ-2": 30,
            "17 ПМИ": 30,
            "17 ПИ": 30,
            "17 ФМ": 10,
        }
        
        self.MODULES_TIMETABLES = {} # расписания всех модулей университета
        
    # =============================== [ Расписание ] ===============================
    class ModuleTimetable(Utils):
    # __________ [ ModuleTimetable ] __________
        # ******************** [ Инициализация ] ********************
        def __init__(self, university,
                module_timetable_number = 3, # по умолчанию для третьего модуля расписание
                max_pairs_amount = 6,
                study_days_per_week_amount = 6,
                module_weeks_amount = 11,
                prefer_max_pairs_per_day = 3,
                prefer_min_pairs_per_day = 2
            ):
            # super()

            self.university = university
            self.university.MODULES_TIMETABLES.update(
                # установим ссылку на текущее расписание для модуля, под указанным номером
                # переменной module_timetable_number
                module_timetable_number = self
            )
            # -------------------- [ Хранилище/Состояния ]
            # self.STORAGE_TimeTable = [] # состояние текущего расписания
            self.STORAGE_ALL_SUBJECTS = [] # список всех уникальных предметов всех групп в расписании
            self.STORAGE_ALL_GROUP_NAMES = [] # список всех уникальных наименований групп

            self.GROUPS = {} # Ссылка на ту илиную группу. Формат объекта: { GroupName: GroupLink }
            self.CURRENT_GROUP = None # Ссылка на текущую рассматриваемую группу

            # Учебный план
            self.EDU_PLAN = {
                "16БИ-2": {
                    'Академическое письмо' : 12,
                    'Методы машинного обучения в информационной безопасности' : 21,
                    'Имитационное моделирование' : 10,
                    'Анализ требований и проектирование информационных систем' : 24
                },
                "16ПМИ": {
                    'Философия науки' : 1,
                    'Случайные процессы' : 12,
                    'Научный семинар' : 5,
                    'Академическое письмо' : 6,
                    'Компьютерная лингвистика' : 22,
                    'Практический курс Интернет вещей' : 22
                },
                "16ПИ": {
                    'Экономика программной инженерии' : 22,
                    'Академическое письмо' : 22,
                    'НИС' : 22,
                    'Технологии IоТ' : 22
                },
                "16ФМ": {
                    'Теория управления' : 22,
                    'Теория вероятностей' : 22,
                    'Вычислительная математика' : 22,
                    'Академическое письмо' : 22,
                    'Общая физика' : 22,
                    'Компьютерная топология' : 22,
                    'Практикум по компьютерной топологии' : 22
                }
            }

            # -------------------- [ Параметры ]
            self.PARAM_MAX_PAIRS_AMOUNT = max_pairs_amount # максимально возможное количество пар в день у расписания
            self.PARAM_STUDY_DAYS_PER_WEEK_AMOUNT = study_days_per_week_amount # количество учебных дней в неделю у расписания
            self.PARAM_MODULE_WEEKS_AMOUNT = module_weeks_amount # количество недель в рассматриваемом модуле расписания
            self.PARAM_PREFER_MAX_PAIRS_PER_DAY = prefer_max_pairs_per_day
            self.PARAM_PREFER_MIN_PAIRS_PER_DAY = prefer_min_pairs_per_day

            self.HARD_CONSTRAINTS_NAMES = [
                "pairs_amount_per_day",
                "pairs_amount_per_week",
                "pairs_in_the_same_corpuses",
                "auditorium_satisfies_pair_type",
                "subject_pairs_amount_per_eduPlan"
            ]
            self.FINES = {
                "big_boss_fine": 100000, # Константный штраф за строгие ограничения
                "pairs_amount_per_day": 1, # [HARD] Количество пар в день у любой группы не должно превышать заданного количества
                "pairs_amount_per_week": 1, # [HARD] Количество пар в неделю у любой группы не должно превышать заданного количества
                "pairs_in_the_same_corpuses": 1, # [HARD] Запрещены переезды между корпусами в течение дня
                "auditorium_satisfies_pair_type": 1, # [HARD] Тип аудитории должен подходить для типа занятия
                "subject_pairs_amount_per_eduPlan": 1, # [HARD] Количество пар конкретного предмета в расписании должно соответствовать их количеству в учебном плане
                
                "pairs_chaining_satisfies": 1, # Желательно, чтобы одно занятие располагалось после другого, если такая связь указана для двух занятий
                "windows_amount_per_day": 1, # Желательно, чтобы количество окон сводилось к минимуму
                "preferable_max_pairs_per_day": 1, # Желательно, чтобы количество пар в день у любой учебной группы не превышало заданного значения
                "preferable_min_pairs_per_day": 1, # Желательно, чтобы количество пар в день у любой учебной группы было больше или равно заданного значения или же равно нулю
                "the_same_pairs_position_by_week": 1, # Желательно, чтобы пары одного и того же предмета проводились в одни и те же дни и временные интервалы каждую неделю в одних и тех же аудиториях
                "changing_auditoriums_per_day_amount": 1, # Желательно, чтобы количество различных предметов в день у любой учебной группы превышало или было равно количеству аудиторий, в которых проводятся эти самые занятия в течение дня
                "saturday_pairs_amount": 1, # Желательно, чтобы количество пар у любой учебной группы в течение субботы сводилось к минимуму
                "less_preferable_pair_timeslots": 1, # Желательно, чтобы в течение дня пары у любой учебной группы располагались в заданном заранее промежутке
                "week_max_particular_subject_pairs_amount": 1 # unkn[OWN]. Желательно, чтобы количество пар в неделю не превышало (pairs_amount_by_EDU_PLAN // PARAM_MODULE_WEEKS_AMOUNT)
            }

            # -------------------- [ Структуры для хранения данных ]
            # Все расписание в виде JSON
            self.FULL_JSON_DATA = None
            # Дни
            self.INDEX_TO_DAYS_DICT = {}
            self.DAYS_TO_INDEX_DICT = {}
            # Тайм-слоты
            self.INDEX_TO_TIME_SLOTS_DICT = {}
            self.TIME_SLOTS_TO_INDEX_DICT = {}
            # Дисциплины
            self.SUBJECT_TO_INDEX_DICT = {}
            self.INDEX_TO_SUBJECT_DICT = {}
            # Пары/Дни/Недели/Расписание
            # self.SCHEDULE_MODULE_ARR = []

            self.PAIRS_CHAIN = {}

            # -------------------- [ Текстовые константы ]
            self.CONST_SUBJ = "Дисциплина"
            self.CONST_TUTOR = "Преподаватель"
            self.CONST_ROOM = "Аудитория"
            self.CONST_LTYPE = "Тип"
            self.CONST_EXTRA = "Дополнительно"
            self.CONST_CAMPUS = "Кампус"
            self.CONST_LECTURE = "Лекция"
            self.CONST_PRACTICE = "Семинар"

            self.DICTIONARY = {
                # Degrees
                "DEG_BAK": "Бакалавриат",
                "DEG_MAG": "Магистратура",

                # Edu Programs
                "EDU_PMI": "Прикладная математика и информатика",
                "EDU_BI": "Бизнес-информатика",
                "EDU_PI": "Программная инженерия",
                "EDU_FM": "Математика",

                # Courses
                "COURSE_1": "1 курс",
                "COURSE_2": "2 курс",
                "COURSE_3": "3 курс",
                "COURSE_4": "4 курс"
            }

            # "Негативные" значения (которые не стоит брать во внимание):
            self.CONST_NEGATIVE = '-'
            self.CONST_NEGATIVE_PAIR = 'Нет пары'
            self.CONST_NEGATIVE_PAIR_INT = 0
            self.CONST_NEGATIVE_GENERAL = 'ОБЩЕЕ'
            self.CONST_NEGATIVE_INDIVIDUAL = "Индивидуально"
            self.CONST_NEGATIVE_SPECIFYING = "Уточняйте у учебного отдела или по расписанию которое было выслано для этой дисциплины"
            self.BUG_SUBJECT = 'Практический курсИнтернет вещей'
            self.NEGATIVE_VALUES = [self.CONST_NEGATIVE_PAIR, self.CONST_NEGATIVE, self.CONST_NEGATIVE_GENERAL, self.CONST_NEGATIVE_INDIVIDUAL, self.CONST_NEGATIVE_SPECIFYING, self.BUG_SUBJECT] #, self.CONST_NEGATIVE_DAY]


            self.SEVERAL_WEEK = False # если расписание поделено на недели
            # -------------------- [ Статистика за модуль ]
            self.MODULE_STATISTICS = {
                'CORPUS_PER_DAY_CHANGED': 0, # сколько было переездов между корпусами в течение дня (за весь модуль)
                'EXCESS_AUDITORIUMS_FULLNESS': 0, # сколько было превышений вмещаемости студентов в аудитории (за весь модуль)
            }

        # ********************** [ Структурирование данных ] ***********************
        def struct_data(self, subjects_arr, days_arr, time_slots_arr, debug = False):
            # print("\n\n\n\n\n\n self.CURRENT_GROUP:", self.CURRENT_GROUP.NAME, "\n\n\n\n\n\n")

            index_to_days_dict = {}
            days_to_index_dict = {}
            # Дисциплины
            subject_to_index_dict = {}
            index_to_subject_dict = {}
            # Тайм-слоты
            time_slots_to_index_dict = {}
            index_to_time_slots_dict = {}

            # Sorting & structing
            for i, day in enumerate(days_arr):
                index = i + 1
                days_to_index_dict.update({ day: index })
                index_to_days_dict.update({ index: day })

            for i, group in enumerate(subjects_arr):
                subject_to_index_dict.update({ group: {} })
                index_to_subject_dict.update({ group: {} })
                for sID, subj in enumerate(subjects_arr[group]):
                    index = sID + 1
                    subject_to_index_dict[group].update({ subj: index })
                    index_to_subject_dict[group].update({ index: subj })

            for i, time_name in enumerate(time_slots_arr):
                index = i + 1
                time_slots_to_index_dict.update({ time_name: index })
                index_to_time_slots_dict.update({ index: time_name })

            if debug:
                # Дни
                # print("days_to_index_dict:")
                # for key, val in days_to_index_dict.items():
                #     print("\t", key, ":", val)

                print("index_to_days_dict / days_to_index_dict:")
                for key, val in index_to_days_dict.items():
                    print("\t", key, ":", val)

                print()

                print("index_to_subject_dict / subject_to_index_dict:")
                # print("index_to_subject_dict:", index_to_subject_dict)
                for group in index_to_subject_dict:
                    # print("\tindex_to_subject_dict[group]:", index_to_subject_dict[group])
                    # print("\tindex_to_subject_dict[group].items():", index_to_subject_dict[group].items())
                    for key, val in index_to_subject_dict[group].items():
                        print("\t", group, "\t", key, ":", val)

                print()

                print("index_to_time_slots_dict / time_slots_to_index_dict:")
                for key, val in index_to_time_slots_dict.items():
                    print("\t", key, ":", val)

            return subject_to_index_dict, index_to_subject_dict, days_to_index_dict, index_to_days_dict, time_slots_to_index_dict, index_to_time_slots_dict

        # ******************** [ Подготовка данных к работе ] ********************
        def prepare_data(self,
            tt_json_file = 'Timetable_ALL_final.json',
            need_particular_info = False, # нужны ли конкретные данные из JSON файла (т.е брать ли во внимание аргументы типа particular_*)
            particular_degree = [], # 'Бакалавриат'
            particular_program = [], # 'Прикладная математика и информатика',
            particular_course = [], # '3 курс'
            particular_group = [], # '17ПМИ'
            debug = False
        ):
            
            # ******************** [ Загрузка данных расписания ] ********************
            def load_from_json(tt_json_file):
                ''' 
                    Загрузка расписания из ранее созданного файла JSON
                    (созданного в свою очередь посредством парсинга расписания из excel документа)
                '''

                # Если файл существует
                if os.path.isfile(tt_json_file):
                    with open(tt_json_file, "r", encoding="utf-8") as file:
                        return json.load(file)
                return None

            # ******************** [ Извлечение данных из файла ] ********************
            def extract_json_data(json_data):
                ''' 
                    Извлечение данных из JSON файла
                    и формирование их в структурированном виде
                '''

                ALL_SUBJECTS_BY_GROUPS = {}
                ALL_TIMES = []
                times_arr_fulled = False
                DAYS_ARR = []

                # ==================== [Метод выборочного парсинга] ====================
                def particular_check(prop, needed_particular_info_arr): # neededValue):
                    if need_particular_info == True:
                        if len(needed_particular_info_arr) > 0: # если есть вообще в чем искать значение
                            if prop not in needed_particular_info_arr: # if prop != neededValue:
                                return False # Если не содержится, вернем False (что будет означать что нужно пропустить итерацию)
                            return True # Выходит, что если содержится, вернет True (что будет означать что можно продолжать делать дальше необходимые действия)
                    return None # Вернет, когда мы не ищем и не рассматриваем какую-то конкретику, либо не с чем сравнивать (пустой массив пришел на вход)


                for degree in json_data: # пройдем по всем степеням обучения (бак, маг)
                    # ===== [БЛОК С СТЕПЕНЬЮ ОБУЧЕНИЯ [START]] =====
                    if False is particular_check(degree, particular_degree): # Если мы рассматриваем не ту степень обучения, что нам нужна, пропустим итерацию
                        continue

                    for edProgram in json_data[degree]: # пройдем по всем образовательным программам
                        # ===== [БЛОК С ОБРАЗОВАТ.ПРОГРАММАМИ [START]] =====
                        if False is particular_check(edProgram, particular_program): # Если мы рассматриваем не ту программу, что нам нужна, пропустим итерацию
                            continue

                        for course in json_data[degree][edProgram]: # пройдем по всем курсам (1-4)
                            # ===== [БЛОК С КУРСАМИ [START]] =====
                            if False is particular_check(course, particular_course): # Если мы рассматриваем не тот курс, что нам нужен, пропустим итерацию
                                continue

                            for group in json_data[degree][edProgram][course]: # пройдем по всем группам
                                # ===== [БЛОК С ГРУППАМИ [START]] =====
                                if False is particular_check(group, particular_group): # Если мы рассматриваем не ту группу, что нам нужна, пропустим итерацию
                                    continue

                                self.STORAGE_ALL_GROUP_NAMES.append(group) # добавляем группу, которую взяли на рассмотрение и не пропустили условием выше

                                ALL_SUBJECTS = []
                                temp_days_arr = []
                                for day in json_data[degree][edProgram][course][group]: # пройдем по всем дням
                                    # ===== [БЛОК С ДНЯМИ [START]] =====
                                    if day in self.NEGATIVE_VALUES:
                                        continue
                                    DAYS_ARR.append(day)

                                    temp_pairs_arr = []
                                    
                                    for time in json_data[degree][edProgram][course][group][day]: # пройдем по всем часам
                                        temp_subjects = [] # возможно дубликат, но уже лишь бы работало
                                        temp_teachers = []

                                        # ===== [БЛОК С ВРЕМЕНЕМ [START]] =====
                                        if times_arr_fulled == False:
                                            ALL_TIMES.append(time)

                                        if len(json_data[degree][edProgram][course][group][day][time][self.CONST_SUBJ]) > 1:
                                            self.SEVERAL_WEEK = True

                                        # Пройдемся по преподавателям
                                        for teacher in json_data[degree][edProgram][course][group][day][time][self.CONST_TUTOR]:
                                            if teacher in self.NEGATIVE_VALUES:
                                                continue
                                            temp_teachers.append(teacher) # добавим преподавателей, которые могут вести текущее занятие
                                        
                                        # Пройдемся по предметам
                                        for subj in json_data[degree][edProgram][course][group][day][time][self.CONST_SUBJ]:
                                            if subj in self.NEGATIVE_VALUES:
                                                continue
                                            temp_days_arr.append(subj)
                                            temp_subjects.append(subj)
                                            # Попытаемся обновить информацию о преподаватетях за текущий предмет (кто его может вести)
                                            try:
                                                # Если удалось:
                                                updateObject(self.university.DB_SUBJECT_TEACHER, {
                                                    subj: list(set(*self.university.DB_SUBJECT_TEACHER[subj], temp_teachers))
                                                })
                                            except:
                                                # Если не удалось:
                                                if subj == 'майнор':
                                                    temp_teachers.append(self.CONST_NEGATIVE_INDIVIDUAL)

                                                updateObject(self.university.DB_SUBJECT_TEACHER, {
                                                    subj: list(set(temp_teachers)) # subj: copy.deepcopy(json_data[degree][edProgram][course][group][day][time][self.CONST_TUTOR])
                                                })

                                        
                                        # for info in json_data[degree][edProgram][course][group][day][time]: # пройдем по всем часам
                                            # ===== [БЛОК С ИНФОРМАЦИЕЙ [START]] =====
                                            # break
                                            # ===== [БЛОК С ИНФОРМАЦИЕЙ [END]] =====
                                        # break
                                        # ===== [БЛОК С ВРЕМЕНЕМ [END]] =====
                                    # ALL_TIMES = list(set(ALL_TIMES))
                                    times_arr_fulled = True
                                    
                                    for subjResult in list(set(temp_days_arr)):
                                        ALL_SUBJECTS.append(subjResult)
                                    
                                    ALL_SUBJECTS = list(set(ALL_SUBJECTS))
                                    # break
                                    # ===== [БЛОК С ДНЯМИ [END]] =====
                                
                                # break
                                ALL_SUBJECTS_BY_GROUPS.update({
                                    group: ALL_SUBJECTS
                                })
                                # ===== [БЛОК С ГРУППАМИ [END]] =====
                            if debug:
                                print("self.university.DB_SUBJECT_TEACHER:", self.university.DB_SUBJECT_TEACHER)
                                print("Current info:", group, degree, edProgram, course)
                            self.STORAGE_ALL_GROUP_NAMES = list(set(self.STORAGE_ALL_GROUP_NAMES)) # создаем список с уникальными значениями на всякий случай
                            # ========== [Создание групп и наполнение их мета-данными] =========
                            for group_name in self.STORAGE_ALL_GROUP_NAMES:
                                if group_name not in self.GROUPS:
                                    self.GROUPS.update(
                                        {
                                            group_name: {
                                                "degree": degree,
                                                "edu_program": edProgram,
                                                "course": course
                                            }
                                        }
                                    )
                            
                            # break # обрываем цикл с курсами
                            # ===== [БЛОК С КУРСАМИ [END]] =====
                        
                        # break # обрываем цикл с образоват.программами
                        # ===== [БЛОК С ОБРАЗОВАТ.ПРОГРАММАМИ [END]] =====
                    
                    # break
                    # ===== [БЛОК С СТЕПЕНЬЮ ОБУЧЕНИЯ [END]] =====

                if len(self.GROUPS) <= 0:
                    raise ValueError("Неудалось заполнить группы информацией! (self.GROUPS пуст!)")

                # Проверим, все ли искомые группы удалось найти и извлечь их информацию:
                if need_particular_info and len(particular_group) > 0:
                    for needed_group in particular_group:
                        # Если искомую группу не удалось найти в объекте всех добавленных групп:
                        if needed_group not in self.GROUPS:
                            tip_info = ''
                            # И если какая-либо информация, сужающая круг поиска группы для извлечения данных указана (а именно: particular_degree ИЛИ particular_program ИЛИ particular_course)
                            if len(particular_degree) > 0 or len(particular_program) >0 or len(particular_course) > 0:
                                # то дадим подсказку, на случай если действительно произошла нестыковка при указании информации выше, не соответствующей искомым группам
                                tip_info = "Может попробовать извлечь данные о группах, не указывая информацию о: particular_degree ИЛИ particular_program ИЛИ particular_course ?"
                            
                            # в любом случае выведем ошибку о том что не удалось найти искому группу
                            self.error("Искомая группа: " + str(needed_group) + " не была найдена в self.GROUPS!", tip_info)

            
                print("GROUPS EXTRACTED AND FILLED:")
                print(self.GROUPS)
                print("***********************************************************\n")
                
                return ALL_SUBJECTS_BY_GROUPS,  DAYS_ARR, ALL_TIMES #ALL_SUBJECTS, DAYS_ARR, ALL_TIMES

            try:
                json_data = load_from_json(tt_json_file)
                subjects_arr, days_arr, times_arr = extract_json_data(json_data)
                print("subjects_arr:", subjects_arr)
                
                s_2_i, i_2_s, \
                d_2_i, i_2_d, \
                t_2_i, i_2_t = self.struct_data(subjects_arr, days_arr, times_arr, debug)

                self.FULL_JSON_DATA = json_data
                self.STORAGE_ALL_SUBJECTS = copy.deepcopy(subjects_arr)

                self.SUBJECT_TO_INDEX_DICT = s_2_i
                self.INDEX_TO_SUBJECT_DICT = i_2_s

                self.DAYS_TO_INDEX_DICT = d_2_i
                self.INDEX_TO_DAYS_DICT = i_2_d

                self.TIME_SLOTS_TO_INDEX_DICT = t_2_i
                self.INDEX_TO_TIME_SLOTS_DICT = i_2_t
            
            except FileNotFoundError:
                print("Не получается найти файл:", tt_json_file)

            return json_data, s_2_i, i_2_s, d_2_i, i_2_d
        
        # *********************** [ Выбор текущей группы ] *************************
        def select_group(self, group_name, reDefine = True):
            chosen_group = None

            try:
                chosen_group = self.GROUPS[group_name]["link"] # попытаемся выбрать группу
                
                if reDefine: # если необходимо переопределить STORAGE переменную:
                    self.CURRENT_GROUP = chosen_group # перезапишем ссылку на текущую выбранную группу
            except KeyError as err:
                self.error("Группа: " + str(group_name) + " не может быть выбрана!", "Возможно по причине того, что она отсутствует в self.GROUPS!")
                self.error("Либо же у группы, которую пытаемся выбрать отсутствует поле \"link\". Вот какую ошибку нам удалось получить:", err)

            return chosen_group

        # ******************** [ Добавить группу ] ********************
        def add_group(self, group_name, group_id = -1):

            if group_name in self.GROUPS: # есть ли вообще нужная нам группа в объекте self.GROUPS (иными словами: извлекали ли мы до этого данные об этой группе?)
                if "link" in self.GROUPS[group_name].keys(): # можно было и не проверять, т.к есть элегантный способ .setdefault(keyName, value), но хочется оповестить пользователя в случае чего
                    # raise ValueError("Для группы: " + str(group_name) + " уже имеется своя ссылка!")
                    self.error(
                        "Нельзя добавить группу, для которой уже был ранее создан свой экземпляр класса!",
                        "Для группы: " + str(group_name) + " уже имеется своя ссылка!"
                    )
                else:
                    return self.GROUPS[group_name].setdefault( "link",
                        self.Group(
                            self, # передаем ссылку на ModuleTimeTable
                            self.university,
                            group_name,
                            self.GROUPS[group_name]["course"],
                            self.GROUPS[group_name]["edu_program"],
                            self.GROUPS[group_name]["degree"]
                        )
                    )
            else:
                raise ValueError("Невозможно добавить группу: " + str(group_name) + ", т.к информация о ней отсутсвует в self.GROUPS!")
            
            return False

        # ******************** [ Удалить группу ] ********************
        def del_group(self, group_name, group_id = -1):
            del self.GROUPS[group_name]
            return True

        # ******************** [ Инициализировать все группы ] ********************
        def init__groups_links(self):
            ''' 
                Создадим экземпляры классов для каждой группы и обновим информацию в self.GROUPS,
                воспользовавшись методом add_group
            '''
            all_groups_links_flatten_arr = [] # просто сплошной массив ссылок на группы
            for group_name in self.GROUPS:
                all_groups_links_flatten_arr.append( self.add_group(group_name) )

            return all_groups_links_flatten_arr

########################################################################################################################################################################################
#########################################################################################################################################################################################
#########################################################################################################################################################################################

        # ============ [ Группа, для которой предназначено расписание ] ============
        class Group(Utils):
        # __________ [ Group ] __________
            def __init__(self, timeTable,university,
                # group_id, # Поле id пока решено убрать:
                group_name,
                group_course = '',
                group_educational_program = '',
                group_degree = ''
            ):
                self.university = university
                self.timeTable = timeTable # ссылка на класс ModuleTimeTable
                #-------------------------
                # self.ID = group_id # Поле id пока решено убрать:
                self.NAME = group_name
                self.COURSE = group_course
                self.EDU_PROGRAM = group_educational_program
                self.DEGREE = group_degree

                self.EDU_PLAN = self.timeTable.EDU_PLAN[self.NAME] # выбор учебного плана по наименованию группы
                self.ALL_SUBJECTS = [ subject_name for subject_name in self.EDU_PLAN ] # наименование всех предметов для конкретной группы

                self.WEEKS = [] # список добавленных учебных недель
                # НазваниеПары_СсылкаНаОбъектПары (для быстрого доступа к объекту по названию пары):
                self.PAIR_LINK = {} # при изменении структуры - изменить везде!

                self.STATISTICS = {}
            
            # ******************** [ Добавить неделю ] ********************
            def add_week(self, w_type = "normal", even_odd = []):
                '''
                    Параметр: even_odd нужен для того, чтобы определить как именовать
                    неделю четную и как именовать неделю нечетную
                    (
                        под индексом 0: наименование четной недели;
                        под индексом 1: наименование нечетной недели
                    )
                '''

                # Если количество добавленных недель уже превышает количество возможных:
                if len(self.WEEKS) >= self.timeTable.PARAM_MODULE_WEEKS_AMOUNT:
                    self.error(
                        "Неделя не может быть добавлена!",
                        "поскольку достигнут лимит максимально возможного количества недель в модуле!"
                    )
                    return False
                
                current_week_number = len(self.WEEKS) + 1 # текущий номер для новой недели = количество имеющихся недель + 1

                if len(even_odd) == 2: # если данный массив наполнен наименованиями недель
                    if current_week_number % 2 == 0: # even
                        w_type = even_odd[0] # even name
                    else:
                        w_type = even_odd[1] # odd name

                self.WEEKS.append(
                    self.Week(
                        # передаем ссылку на класс: ModuleTimetable
                        self.timeTable, # этот self = timeTable внутри Week, чтобы можно было использовать класс ModuleTimetable внутри Week
                        self, # а также передаем ссылку на группу, для которой эта учебная неделя создавалась
                        self.university,
                        current_week_number,
                        w_type
                    )
                )
                return True

            # ******************** [ Удалить неделю ] ********************
            def del_week(self, w_number = -1, rename_by_even_odd = False):
                del self.WEEKS[w_number]

                # заново пронумеруем недели, если была удалена конкретная неделя, а не последняя в списке
                if w_number != -1:
                    for i, week in enumerate(self.WEEKS):
                        week.NUM = i + 1

                        # если есть необходимость в переименовывании в порядке even_odd
                        if rename_by_even_odd:
                            even_odd = ['up', 'down']

                            if (i + 1) % 2 == 0: # even
                                week.TYPE = even_odd[0] # even name
                            else:
                                week.TYPE = even_odd[1] # odd name
                
                return True

            # ******************** [ Инициализировать недели ] ********************
            def init__empty_weeks(self, debug = False):
                if debug: print("Adding weeks into the ModuleTimeTable...")
                for i in range(self.timeTable.PARAM_MODULE_WEEKS_AMOUNT):
                    if self.add_week():
                        if debug: print("\tWeek", i, "was initialized")
                    else:
                        break

            # ************** [ Инициализировать все дни (и пустые пары, если необходимо) во всех неделях ] **************
            def init__weeks__empty_days(self, init_time_slots = False, debug = False): # def init_days_each_week(self, debug = False):
                for currentWeek in self.WEEKS:
                    currentWeek.init_days(debug = debug)

                    if init_time_slots:
                        for currentDay in currentWeek.DAYS:
                            currentDay.init_time_slots(debug = debug)

            # ************** [ Инициализировать всё ] **************
            def init__empty_all(self, debug = False):
                self.init__empty_weeks(debug = debug)
                self.init__weeks__empty_days(init_time_slots = True, debug = debug)
                self.reset_stats()

                if debug: print("Недели, Дни, Тайм-слоты были успешно проинициализированы для группы:", self.NAME)
                return True
            
            # ************** [ Инициализировать/Сбросить всю статистику ] **************
            def reset_stats(self):
                self.STATISTICS = {
                    # KEY = ограничение по которому ведется статистика: VALUE = [ количество_этого_нарушения, общая_накопившеяся_величина_нарушения ]
                    "pairs_amount_per_day": [0, 0],
                    "pairs_amount_per_week": [0, 0],
                    "pairs_in_the_same_corpuses": [0, 0],
                    "auditorium_satisfies_pair_type": [0, 0],
                    "subject_pairs_amount_per_eduPlan": [0, 0],
                    "pairs_chaining_satisfies": [0, 0],
                    "windows_amount_per_day": [0, 0],
                    "preferable_max_pairs_per_day": [0, 0],
                    "preferable_min_pairs_per_day": [0, 0],
                    "the_same_pairs_position_by_week": [0, 0],
                    "changing_auditoriums_per_day_amount": [0, 0],
                    "saturday_pairs_amount": [0, 0],
                    "less_preferable_pair_timeslots": [0, 0],
                    "week_max_particular_subject_pairs_amount": [0, 0],

                    "WHOLE_GENERAL_FITNESS": 0 # общий накопленный фитнесс для текущей группы по всем нарушениям
                }

            # ======== [ Функция для заполнения группы данными из json файла ] ========
            def fill_data_from_timetable(self):
                needed_data = copy.deepcopy(self.timeTable.FULL_JSON_DATA[self.DEGREE][self.EDU_PROGRAM][self.COURSE][self.NAME])
                del needed_data['ОБЩЕЕ']


                for weekIDX, week in enumerate(self.WEEKS): # пройдем по всем неделям
                    for dayIDX, day in enumerate(needed_data): # пройдем по всем дням
                        for timeIDX, time in enumerate(needed_data[day]): # пройдем по всем time-slot'ам
                            for subject in needed_data[day][time][self.timeTable.CONST_SUBJ]:
                            
                                # если дисциплина из разряда NEGATIVE_VALUES (нет пары, символ "-" и т.п)
                                if subject in self.timeTable.NEGATIVE_VALUES:
                                    continue # пропускаем итерацию

                                if timeIDX > 0: # если пару можно сравнивать с предыдущей в time-slot'е
                                    # если одинаковое название у пар (предыдущей и текщуей), то просто копируем ее, путем копирования ссылки на нее
                                    # upd: НО ТОЛЬКО ЕСЛИ ОНИ ОТЛИЧАЮТСЯ ТИПОМ (ПРАКТИКА/ЛЕКЦИЯ) - ТО ЭТО ВСЕ ЕЩЕ ОТДЕЛЬНЫЕ ПАРЫ
                                    if week.DAYS[dayIDX].isPair(week.DAYS[dayIDX].PAIRS[timeIDX - 1]):
                                        if week.DAYS[dayIDX].PAIRS[timeIDX - 1].SUBJECT_NAME == subject \
                                            and week.DAYS[dayIDX].PAIRS[timeIDX - 1].TYPE == needed_data[day][time][self.timeTable.CONST_LTYPE]:
                                            week.DAYS[dayIDX].PAIRS[timeIDX] = week.DAYS[dayIDX].PAIRS[timeIDX - 1]
                                            continue

                                if weekIDX > 0: # если текущую неделю можно сравнивать с предыдущей, то тоже копируем ссылку на предмет в текущий день, текущее время с прошлой недели
                                    if self.WEEKS[weekIDX - 1].DAYS[dayIDX].PAIRS[timeIDX].SUBJECT_NAME == subject:
                                        week.DAYS[dayIDX].PAIRS[timeIDX] = self.WEEKS[weekIDX - 1].DAYS[dayIDX].PAIRS[timeIDX]
                                        continue

                                week.DAYS[dayIDX].add_pair(
                                    time_slot_ID = timeIDX,
                                    name = subject, # пока что берем принудительно одну пару, а не весь массив
                                    tutors = needed_data[day][time][self.timeTable.CONST_TUTOR],
                                    rooms = needed_data[day][time][self.timeTable.CONST_ROOM],
                                    pairType = needed_data[day][time][self.timeTable.CONST_LTYPE], # на вход приходит строка
                                    extra = needed_data[day][time][self.timeTable.CONST_EXTRA], # на вход приходит строка
                                    corpus = needed_data[day][time][self.timeTable.CONST_CAMPUS], # на вход приходит строка, а не массив корпусов
                                    
                                    # Тут можно было предварительно сделать проверку на то у каких еще групп этот предмет ведется
                                    # и если есть еще таковая группа, то добавить сюда, а пока просто та, для которой данные и заполняются
                                    groups = [self.NAME]
                                )

                        week.DAYS[dayIDX].transform_pairs_to_int_arr()

            # ======== [ Функция для визуализации расписания в виде массивов ] ========
            def visualize(self, printResult = True, print_subj_to_idx = True, returnResult = False):
                if print_subj_to_idx:
                    print(self.timeTable.SUBJECT_TO_INDEX_DICT[self.NAME])
                
                weeks_visualizations = []
                for week in self.WEEKS:
                    weeks_visualizations.append( week.visualize(printResult = printResult, returnResult = True) )

                if returnResult:
                    return weeks_visualizations

            # ======== [ Функция получения свободной ячейки рандомно ] ========
            def get_random_free_cell(self, maximum_attempts = 200):
                isFree = False
                max_tryings = maximum_attempts

                while isFree == False:
                    if max_tryings <= 0:
                        break
                        raise Exception('Cant find a free cell!!!')

                    # random.seed(6)
                    randWeekID = random.randint(0, len(self.WEEKS) - 1)
                    randDayID = random.randint(0, len(self.WEEKS[randWeekID].DAYS) - 1)
                    randTimeSlotID = random.randint(0, len(self.WEEKS[randWeekID].DAYS[randDayID].PAIRS) - 1)
                    
                    if self.WEEKS[randWeekID].DAYS[randDayID].PAIRS[randTimeSlotID] is self.timeTable.CONST_NEGATIVE_PAIR_INT:
                        isFree = True
                        return randWeekID, randDayID, randTimeSlotID
                    else:
                        max_tryings -= 1
                return randWeekID, randDayID, randTimeSlotID
            
            # ======== [ Функция получения рандомной ячейки с парой ] ========
            def get_random_pair_cell(self, maximum_attempts = 200):
                isFound = False
                max_tryings = maximum_attempts

                while isFound == False:
                    if max_tryings <= 0:
                        break
                        raise Exception('Cant find a pair cell!!!')

                    # random.seed(6)
                    randWeekID = random.randint(0, len(self.WEEKS) - 1)
                    randDayID = random.randint(0, len(self.WEEKS[randWeekID].DAYS) - 1)
                    randTimeSlotID = random.randint(0, len(self.WEEKS[randWeekID].DAYS[randDayID].PAIRS) - 1)
                    
                    if self.WEEKS[randWeekID].DAYS[randDayID].isPair(self.WEEKS[randWeekID].DAYS[randDayID].PAIRS[randTimeSlotID]):
                        isFound = True
                        return randWeekID, randDayID, randTimeSlotID
                    else:
                        max_tryings -= 1
                return randWeekID, randDayID, randTimeSlotID

            # ======== [ Функция сброса всего расписания для группы ] ========
            def factory_reset(self):
                # for weekID in range(len(self.WEEKS)):
                #     for dayID in range(len(self.WEEKS[weekID].DAYS)):
                #         for pairID in range(len(self.WEEKS[weekID].DAYS[dayID].PAIRS)):
                #             del self.WEEKS[weekID].DAYS[dayID].PAIRS[pairID]
                #         del self.WEEKS[weekID].DAYS[dayID]
                #     del self.WEEKS[weekID]
                
                        
                self.WEEKS = []
                self.PAIR_LINK.clear()
                self.reset_stats()
                self.init__empty_all()
                # print("\t factory_reset:", self.PAIR_LINK)

            # ======== [ Функция выявления наличия нарушенных жестких ограничений ] ========
            def has_hard_constraints(self):
                has_hard = False

                # пройдемся по названиям всех hard ограничений:
                for current_constraint in self.timeTable.HARD_CONSTRAINTS_NAMES:
                    if self.STATISTICS[current_constraint][0] > 0: # если количество нарушений жесткого ограничения превышает 0
                        has_hard = True # значит такое нарушение имеется
                        break # останавливаем цикл с поиском

                return has_hard # возвращаем результат
            
            # ======== [ Функция рандомного поиска ] ========
            def random_search(self):
                rerun = False # необходимость перезапуска по умолчанию

                self.factory_reset() # сброс расписания
                # print("\t\t BEFORE random_search:", self.PAIR_LINK)
                self.random_timetable() # составление рандомного расписания
                # print("\t\t AFTER random_search:", self.PAIR_LINK)
                self.timeTable.calc_fitness() # подсчет фитнесса
                rerun = self.has_hard_constraints() # проверка на наличие жестких нарушений
                # (результатом проверки будет решение о необходимости перезапуска составления расписания)
            
                
                return rerun

            # ======== [ Функция-обертка запуска рандомного поиска ] ========
            def launch_random_search(self, total_attempts = 100):
                curr_attempt = 0

                rerun = True
                # result = []
                while rerun == True and curr_attempt < total_attempts:
                    rerun = self.random_search()

                    if rerun:
                        curr_attempt += 1
                        self.random_search()
                    else:
                        # _, result = self.timeTable.calc_fitness()
                        self.timeTable.calc_fitness()
                
                # print("\t\t\t launch_random_search:", self.PAIR_LINK)
                return self.STATISTICS, curr_attempt # return result, curr_attempt


            # ======== [ Функция составления рандомного расписания ] ========
            def random_timetable(self):
                CREATED_PAIRS_LINKS = {}
                for subj in self.EDU_PLAN:
                    # print("Subj:", subj)
                    for subj_number in range(self.EDU_PLAN[subj]):
                        # print(subj_number, "/", self.EDU_PLAN[subj])
                        randWeekID, randDayID, randTimeSlotID = self.get_random_free_cell()
                        # print("randWeekID:", randWeekID, "| randDayID:", randDayID, "| randTimeSlotID:", randTimeSlotID)

                        link = None
                        try:
                            link = CREATED_PAIRS_LINKS[subj]
                            self.WEEKS[randWeekID].DAYS[randDayID].PAIRS[randTimeSlotID] = link
                        except:
                            chosen_corpus = random.choice(self.university.DB_CORPUSES)
                            # print("\n\nIS IT EMPTY?\n\n")
                            # print(self.university.DB_SUBJECT_TEACHER)
                            # print("subj:", subj)
                            # print("self.university.DB_SUBJECT_TEACHER[subj]:", self.university.DB_SUBJECT_TEACHER[subj])
                            
                            self.WEEKS[randWeekID].DAYS[randDayID].add_pair(
                                time_slot_ID = randTimeSlotID,
                                name = subj,
                                tutors = random.choice(list( self.university.DB_SUBJECT_TEACHER[subj] )),
                                rooms = [random.choice( list( self.university.DB_AUDITORIUM[ chosen_corpus ] ) )],
                                pairType = random.choice(['Лекция', 'Семинар']),
                                extra = '',
                                corpus = chosen_corpus,
                                groups = [self.NAME]
                            )
                            link = self.WEEKS[randWeekID].DAYS[randDayID].PAIRS[randTimeSlotID]
                            CREATED_PAIRS_LINKS.update({ subj: link })
 
            # !!!!!!!!!!!!!!!!!!!! [ Жесткие ограничения ] !!!!!!!!!!!!!!!!!!!!
            def CHECK_STRICT_CONSTRAINT_subject_pairs_amount_per_eduPlan(self, subject_name, exception_arr=['майнор'], debug = False):
                '''
                    Количество пар конкретного предмета в расписании должно соответствовать их количеству в учебном плане
                '''
                if subject_name in exception_arr: return True # игнорируем

                needed_subject_amount = self.EDU_PLAN[subject_name]
                subject_amount_in_fact = 0

                if debug: print("needed_subject_amount:",needed_subject_amount)

                for i, week in enumerate(self.WEEKS):
                    # if debug: print("WEEK", i)
                    for day in week.DAYS:
                        subject_amount_in_fact += day.get_pair_amount_by_link(self.PAIR_LINK[subject_name], debug = debug)

                if debug: print("subject_amount_in_fact:", subject_amount_in_fact)

                if needed_subject_amount != subject_amount_in_fact:
                    if debug: print("THE DIFFERENCE IS:", needed_subject_amount - subject_amount_in_fact)
                    return False # not satisfied

                return True # satisfied

            def LAUNCH_CHECK_STRICT_CONSTRAINT_subject_pairs_amount_per_eduPlan(self):
                condition_satisfied = True
                fitness_value = 0

                for subj_name in self.EDU_PLAN:
                    
                    # Количество пар конкретного предмета в расписании должно соответствовать их количеству в учебном плане:
                    if self.CHECK_STRICT_CONSTRAINT_subject_pairs_amount_per_eduPlan(subj_name) == False:
                        condition_satisfied = False
                        fitness_value += self.timeTable.FINES["big_boss_fine"]

                return condition_satisfied, fitness_value

            # ~~~~~~~~~~~~~~~~~~~~ [ Мягкие ограничение ] ~~~~~~~~~~~~~~~~~~~~
            def CHECK_WEAK_CONSTRAINT_the_same_pairs_position_by_week(self):
                '''
                    Желательно, чтобы пары одного и того же предмета проводились в одни и те же дни и временные интервалы каждую неделю в одних и тех же аудиториях
                '''
                
                module_stats = {}
                weeks_stats = {}
                for wID, current_week in enumerate(self.WEEKS):
                    if wID == 0:
                        continue

                    days_stats = {}
                    for dID, current_day in enumerate(current_week.DAYS):
                        
                        pairs_matching_status = []
                        # пройдемся по парам посчитаем сколько несовпадений:
                        for pID, current_pair in enumerate(current_day.PAIRS):
                            # добавим в статистику результат сравнения пар между собой (True - совпадают; False - не совпадают):
                            are_pairs_the_same = current_pair == self.WEEKS[0].DAYS[dID].PAIRS[pID] # узнаем, совпадают ли пары
                            are_pairs_rooms_the_same = None # т.к в current_pair может попасть число 0, у которого не будет свойства .ROOM, то по умолчанию мы можем не рассматривать совпадение их аудиторий

                            # Проверяем текущая пара действительно ли является парой, и у номинальной недели тоже проверяем этот момент, иначе сравнивать будет не с чем
                            if current_day.isPair(current_pair) and current_day.isPair(self.WEEKS[0].DAYS[dID].PAIRS[pID]):
                                are_pairs_rooms_the_same = current_pair.ROOM == self.WEEKS[0].DAYS[dID].PAIRS[pID].ROOM

                            pairs_matching_status.append( # добавляем результат проверки...
                                are_pairs_the_same and # если пары совпадают И...
                                (
                                    ( # если на совпадение аудиторий удалось сравнить, И результат сравнения True:
                                        (are_pairs_rooms_the_same is not None) and are_pairs_rooms_the_same == True
                                    )
                                    # или иначе, если сравнить не удалось и значение are_pairs_rooms_the_same равно None
                                    or
                                    (
                                        are_pairs_rooms_the_same is None
                                    )
                                )
                            )
                        
                        # обновим статистику за этот текущий dID день и запишем:
                        days_stats.update({
                            dID: {
                                'PAIRS': [*pairs_matching_status], # каждый слот массива pairs_matching_status имеет булевое значение, означающее совпадает ли пара в текущем timeslot'е с тем же timeslot'ом в номинальной неделей
                                'MATCHES_AMOUNT': Counter(pairs_matching_status)[True], # посчитаем количество совпадений основываясь на pairs_matching_status
                                'MISMATCHES_AMOUNT': Counter(pairs_matching_status)[False] # посчитаем количество несовпадений основываясь на pairs_matching_status
                            }
                        })
                    # посчитаем статистику за неделю:
                    whole_week_matches_amount = 0
                    whole_week_mismatches_amount = 0

                    for d in days_stats:
                        whole_week_matches_amount += days_stats[d]["MATCHES_AMOUNT"]
                        whole_week_mismatches_amount += days_stats[d]["MISMATCHES_AMOUNT"]


                    # добавим статистику за неделю :
                    weeks_stats.update({
                        wID: {
                            "DAYS_STATS": days_stats,
                            "WHOLE_WEEK_MATCHES": whole_week_matches_amount,
                            "WHOLE_WEEK_MISMATCHES": whole_week_mismatches_amount
                        }
                    })
                
                # посчитаем статистику за модуль:
                whole_module_matches_amount = 0
                whole_module_mismatches_amount = 0

                for w in weeks_stats:
                    whole_module_matches_amount += weeks_stats[w]["WHOLE_WEEK_MATCHES"]
                    whole_module_mismatches_amount += weeks_stats[w]["WHOLE_WEEK_MISMATCHES"]

                
                module_stats.update({
                    "WEEKS_STATS": weeks_stats,
                    "WHOLE_MODULE_MATCHES": whole_module_matches_amount,
                    "WHOLE_MODULE_MISMATCHES": whole_module_mismatches_amount
                })

                return module_stats

            # ----------------------------------------------------------------------
            
########################################################################################################################################################################################
#########################################################################################################################################################################################
#########################################################################################################################################################################################
            # ====================== [ Неделя расписания ] =========================
            class Week(Utils):
            # __________ [ Week ] __________
                # ******************** [ Инициализация ] ********************
                def __init__(self, timeTable, group, university,
                    week_number,
                    week_type="normal" # up/down/normal
                ):
                    self.university = university
                    self.timeTable = timeTable # ссылка на класс ModuleTimeTable
                    self.group = group
                    #-------------------------

                    self.NUM = week_number
                    self.TYPE = week_type
                                
                    self.DAYS = [
                        # [] * self.PARAM_STUDY_DAYS_PER_WEEK_AMOUNT
                    ]  # список добавленных дней

                # ******************** [ Добавить день ] ********************
                def add_day(self, d_name):
                    # Если количество добавленных дней уже превышает количество возможных:
                    if len(self.DAYS) > self.timeTable.PARAM_STUDY_DAYS_PER_WEEK_AMOUNT:
                        self.error(
                            "Учебный день не может быть добавлен!",
                            "поскольку достигнут лимит максимально возможного количества дней в неделю!"
                        )
                        return False
                    
                    self.DAYS.append( self.Day(
                            self.timeTable, # передаем ссылку на класс: ModuleTimetable
                            self.group, # передаем ссылку на класс: Group
                            self, # передаем ссылку на этот класс: Week
                            self.university, 
                            d_name
                        )
                    )
                    return True

                # ******************** [ Удалить день ] ********************
                def del_day(self, d_name = '', d_id = -1):
                    # Удаление дня по его названию
                    if d_name != '':
                        day_has_been_deleted = False
                        for i, day in enumerate(self.DAYS):
                            if day.NAME == d_name: # если название дня совпадает
                                del self.DAYS[i] # удаляем этот день
                                day_has_been_deleted = True # день был успешно удален
                                break
                        
                        return day_has_been_deleted
                    # Удаление дня по его номеру (если d_id = -1, то просто удалится последний день)
                    else:
                        del self.DAYS[d_id]
                        return True

                # ******************** [ Инициализировать дни ] ********************
                def init_days(self, debug = False):
                    if debug: print("Adding days into the Week №", self.NUM, "...")
                    for dayName in self.timeTable.DAYS_TO_INDEX_DICT:
                        if self.add_day(dayName):
                            if debug: print('\tDay:', dayName, 'was initialized')
                        else:
                            break

                # *** [ Задать пары текущей недели в соответствии с массивом массивов числовых значений ] ****
                def set_week_pairs_by_int_arr(self, week_arr_with_pairs_int_arr):
                    # total_attempts = 1
                    # current_attempt = 0
                    # Пройдем по каждому элементу (массив пар в день) всего недельного массива
                    for dayID, dayPairsIntArr in enumerate(week_arr_with_pairs_int_arr):
                        # для каждого дня запустим функцию установления пар в соответствии с массивом из числовых эквивалентов пар
                        succeed_execution, _, __ = self.DAYS[dayID].set_pairs_by_int_arr(dayPairsIntArr)
                        
                        # while succeed_execution is False and current_attempt < total_attempts:
                        if succeed_execution is False:
                            print("\t\tНе удалось установить пары. Попробуем еще раз...") #Попытка №", current_attempt + 1, "/", total_attempts)
                            self.set_week_pairs_by_int_arr( week_arr_with_pairs_int_arr )
                    
                    return self.DAYS # вернем по итогу наш новый массив дней

                # *** [ Визуализация недели в виде массивов с числовыми эквивалентами ] ****
                def visualize(self, printResult = True, returnResult = False):
                    week_arr_result = []
                    for day in self.DAYS:
                        week_arr_result.append( day.visualize(printResult = False, returnResult = True) )
                    
                    if printResult:
                        print("WEEK №" + str(self.NUM))
                        print("\t", week_arr_result)

                    if returnResult:
                        return week_arr_result

                # !!!!!!!!!!!!!!!!!!!! [ Жесткие ограничение ] !!!!!!!!!!!!!!!!!!!!
                def CHECK_WEAK_CONSTRAINT_pairs_amount_per_week(self, param_limit = 18):
                    '''
                        Количество пар в неделю у любой группы не должно превышать заданного количества
                        (НЕ БОЛЬШЕ 18 (3пар * 6дней) )
                    '''
                    general_pairs_amount = 0
                    for day in self.DAYS:
                        general_pairs_amount += day.get_pairs_amount_per_day()

                    if general_pairs_amount > param_limit:
                        return False # not satisfy

                    return True # statisfy

                def LAUNCH_CHECK_WEAK_CONSTRAINT_pairs_amount_per_week(self):
                    condition_satisfied = True
                    fitness_value = 0
                
                    for week in self.group.WEEKS:

                        if week.CHECK_WEAK_CONSTRAINT_pairs_amount_per_week( (3 * self.timeTable.PARAM_STUDY_DAYS_PER_WEEK_AMOUNT) ) == False:  # Худший случай: 3 пары * 6 дней
                            condition_satisfied = False
                            fitness_value += self.timeTable.FINES["big_boss_fine"]

                        # print("_______________________________________________________\n")
                        break # пока один раз посчитаем, т.к остальные недели дублируются

                    return condition_satisfied, fitness_value
                # ----------------------------------------------------------------------
                
                # ~~~~~~~~~~~~~~~~~~~~ [ Мягкие ограничение ] ~~~~~~~~~~~~~~~~~~~~
                def CHECK_WEAK_CONSTRAINT_saturday_pairs_amount(self):
                    '''
                        Желательно, чтобы количество пар у любой учебной группы в течение субботы сводилось к минимуму
                    '''
                    saturday_pairs_amount = self.DAYS[5].get_pairs_amount_per_day() # можно было бы использовать self.DAYS[-1], но чисто на всякий используем 5ый по счету индекс дня недели)
                    
                    if saturday_pairs_amount == 0: # Если пар в субботу вообще нет
                        return True, saturday_pairs_amount
                    
                    return False, saturday_pairs_amount # Иначе, возвращаем не удовлетворение ограничения и само количество пар в субботу

                # ----------------------------------------------------------------------
                def CHECK_WEAK_CONSTRAINT_week_max_particular_subject_pairs_amount(self, subject_name, debug = False):
                    edu_plan_subject_amount = self.timeTable.EDU_PLAN[self.group.NAME][subject_name]
                    max_per_week_subjects_amount = edu_plan_subject_amount // self.timeTable.PARAM_MODULE_WEEKS_AMOUNT
                    current_subject_per_week_amount = 0

                    if debug: print("max_per_week_subjects_amount:",max_per_week_subjects_amount)
                    
                    for day in self.DAYS:
                        current_subject_per_week_amount += day.get_pair_amount_by_link(self.group.PAIR_LINK[subject_name], debug = debug)

                    if debug: print("current_subject_per_week_amount:", current_subject_per_week_amount)

                    if current_subject_per_week_amount > max_per_week_subjects_amount:
                        if debug: print("THE DIFFERENCE IS:", current_subject_per_week_amount - max_per_week_subjects_amount)
                        return False, current_subject_per_week_amount - max_per_week_subjects_amount # not satisfied

                    return True, 0 # satisfied
                # ----------------------------------------------------------------------

#########################################################################################################################################################################################
#########################################################################################################################################################################################
#########################################################################################################################################################################################      
                # =================== [ День недели расписания ] ===================
                class Day(Utils):
                # __________ [ Day ] __________
                    # ******************** [ Инициализация ] ********************
                    def __init__(self, timeTable, group, week, university,
                        day_name
                    ):
                        self.university = university
                        self.timeTable = timeTable
                        self.group = group
                        self.week = week
                        #-------------------------

                        self.NAME = day_name
                        self.PAIRS = []
                        self.PAIRS_INT = []
                        ''' 
                            [ # PAIRS_ARR
                                [ # 8:00-9:20
                                    {
                                        name = 'Дисциплина без названия',
                                        tutor = '-',
                                        room = '-',
                                        type = '-',
                                        extra = '-',
                                        corpus = '-',
                                        group = '!GROUP_1!'
                                    },
                                    {
                                        name = 'Дисциплина без названия',
                                        tutor = '-',
                                        room = '-',
                                        type = '-',
                                        extra = '-',
                                        corpus = '-',
                                        group = '!GROUP_2!'
                                    },
                                    ...
                                ],
                                [ # 9:30-10:50
                                    {
                                        name = 'Дисциплина без названия',
                                        tutor = '-',
                                        room = '-',
                                        type = '-',
                                        extra = '-',
                                        corpus = '-',
                                        group = '!GROUP_1!'
                                    }
                                    ...
                                ]
                            ]
                        '''
                    
                    # ******************** [ Добавить пару ] ********************
                    def add_pair(self,
                        time_slot_ID,
                        name,
                        tutors = [], # tutor = '-',
                        rooms = [], # room = '-',
                        pairType = '',
                        extra = '',
                        corpus = '',
                        groups = [] # group = ''
                    ):
                        # Если количество добавленных пар уже превышает количество возможных:
                        if len(self.PAIRS) > self.timeTable.PARAM_MAX_PAIRS_AMOUNT:
                            self.error(
                                "Пара не может быть добавлена!",
                                "поскольку достигнут лимит максимально возможного количества пар в день!"
                            )
                            return False
                        
                        if self.PAIRS[time_slot_ID] != self.timeTable.CONST_NEGATIVE_PAIR_INT:
                            self.error(
                                "Пара не может быть вставлена в тайм-слот №" + str(time_slot_ID) + "!",
                                "поскольку тайм-слот: " + str(self.timeTable.INDEX_TO_TIME_SLOTS_DICT[time_slot_ID + 1]) + " уже занят другой парой:" + self.PAIRS[time_slot_ID].SUBJECT_NAME
                            )
                            return False

                        else:
                            self.PAIRS[time_slot_ID] = self.Pair(
                                self.timeTable, # передаем ссылку на класс: ModuleTimetable
                                self.group, # передаем ссылку на класс: Group
                                self.week, # передаем ссылку на класс: Week
                                self, # передаем ссылку на этот класс: Day
                                self.university,
                                name,
                                copy.deepcopy(tutors),
                                copy.deepcopy(rooms),
                                pairType,
                                extra,
                                corpus,
                                copy.deepcopy(groups)
                            )

                            self.group.PAIR_LINK.update({ name : self.PAIRS[time_slot_ID] })

                            # для ограничения соответствия рядом стоящих пар лекции и практики:
                            according_data = {
                                # в данный объект запишется либо ключ "Лекция", либо ключ "Практика", в зависимости от типа добавляемой пары (pairType)
                                # в качестве значения будет выступать непосредственно ссылка на саму добавляемую пару
                                pairType: self.PAIRS[time_slot_ID]
                            }

                            # воспользовавшись кастомной функцией update, обновим данные объекта:
                            updateObject(self.timeTable.PAIRS_CHAIN, # обновим информацию цепочки пар
                            {
                                self.group.NAME: { # для текущей группы
                                    name: { # для текущего предмета
                                        **according_data # добавим ссылку на текущую добавляемую пару
                                    }
                                }
                            })

                        return True

                    # ******************** [ Удалить пару ] ********************
                    def del_pair(self, pair_time = '', pair_num = -1):
                        
                        # Удаление пары по ее счету
                        self.PAIRS[pair_num] = 0
                        return True

                    # ******************** [ Переместить пару ] ********************
                    def move_pair(self, pair_num, toward_weekID, toward_dayID, toward_timeSlotID):
                        
                        pair_link = self.PAIRS[pair_num] # сохраним ссылку текущей пары

                        if self.isPair(self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID]): # если в данной ячейке уже находится пара
                            self.error(
                                "Пара не может быть вставлена в тайм-слот №" + str(toward_timeSlotID) + "!",
                                "поскольку тайм-слот: " + str(self.timeTable.INDEX_TO_TIME_SLOTS_DICT[toward_timeSlotID + 1]) + " уже занят другой парой:" + self.PAIRS[pair_num].SUBJECT_NAME
                            )
                            return False
                        
                        # если ошибка выше не произошла, значит место свободно, можно перемещать
                        self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID] = pair_link # устанавливаем в выбранную ячейку нашу пару 
                        
                        # Удаление пары с ее текущего места
                        self.del_pair(pair_num = pair_num)

                        return True

                    # ******************** [ Поменять пары местами ] ********************
                    def swap_pairs(self, pair_num, toward_weekID, toward_dayID, toward_timeSlotID):
                        
                        pair_link_1 = copy.deepcopy(self.PAIRS[pair_num]) # сохраним ссылку текущей пары #1
                        pair_link_2 = copy.deepcopy(self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID]) # сохраним ссылку текущей пары #2

                        if self.isPair(self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID]) is False: # если в данной ячейке находится НЕ пара
                            self.error(
                                "Невозможно поменяться с парой в тайм-слоте №" + str(toward_timeSlotID) + "!",
                                "поскольку тайм-слот: " + str(self.timeTable.INDEX_TO_TIME_SLOTS_DICT[toward_timeSlotID + 1]) + " имеет значение отсутствия пары:" + self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID]
                            )
                            return False
                        
                        # если ошибка выше не произошла, значит можно меняться
                        self.group.WEEKS[toward_weekID].DAYS[toward_dayID].PAIRS[toward_timeSlotID] = pair_link_1 # устанавливаем в выбранную ячейку нашу пару
                        self.PAIRS[pair_num] = pair_link_2 # устанавливаем в выбранную ячейку нашу пару
                        return True


                    # ********** [ Превратить пары в массив числовых значений ] ***********
                    def transform_pairs_to_int_arr(self):
                        day_arr_result = []
                        for pair in self.PAIRS:
                            if self.isPair(pair): #if pair != self.timeTable.CONST_NEGATIVE_PAIR_INT:
                                # print("transform_pairs_to_int_arr":, self.timeTable.SUBJECT_TO_INDEX_DICT[self.group.NAME][ pair.SUBJECT_NAME ])
                                day_arr_result.append(self.timeTable.SUBJECT_TO_INDEX_DICT[self.group.NAME][ pair.SUBJECT_NAME ])
                            else:
                                day_arr_result.append(self.timeTable.CONST_NEGATIVE_PAIR_INT)

                        self.PAIRS_INT = day_arr_result
                        return day_arr_result

                    # *** [ Задать пары текущего дня в соответствии с массивом числовых значений ] ****
                    def set_pairs_by_int_arr(self, int_pairs_arr):
                        # print("\tint_pairs_arr:", int_pairs_arr)
                        succeed_execution = True
                        for timeSlotIDX, pairIDX in enumerate(int_pairs_arr):
                            # print("self.timeTable.INDEX_TO_SUBJECT_DICT[", self.group.NAME, "]", self.timeTable.INDEX_TO_SUBJECT_DICT[self.group.NAME])
                            try: # попробуем извлечь название предмета, и ссылку на него
                                subject_name = self.timeTable.INDEX_TO_SUBJECT_DICT[self.group.NAME][pairIDX] # получим наименование предмета по ID
                                # print("\tsubject_name:", subject_name)
                                # print("\tself.group.PAIR_LINK:", self.group.PAIR_LINK,"\n")
                                subject_link = self.group.PAIR_LINK[subject_name] # получим ссылку на такую пару по названию предмета
                                self.PAIRS[timeSlotIDX] = subject_link # перебьем таймслот новой парой
                            except: # в случае неудачи, скорее всего у нас такой пары нет, либо все-таки было указано как "отсутвтие пары" (CONST_NEGATIVE_PAIR_INT)
                                if pairIDX == self.timeTable.CONST_NEGATIVE_PAIR_INT: # если мы указали  как "отсутвтие пары":
                                    self.PAIRS[timeSlotIDX] = pairIDX # установим это значение в таймслоте
                                
                                elif not self.group.PAIR_LINK: # если у нас не нашлось ссылки на пару (то можно добавить пары по умолчанию пока что хотя бы)
                                    print("\tself.group.PAIR_LINK оказался пустым, проводим \"инициализацию\" и повторяем попытку...")
                                    self.group.fill_data_from_timetable() # пока заполним так чтобы ссылки появились на пары
                                    # self.set_pairs_by_int_arr(int_pairs_arr)
                                    succeed_execution = False
                                    break

                                else: # пары с таким ID не существует вовсе
                                    self.error(
                                        "Неудалось установить пару с ID = " + str(pairIDX) + " для группы " + str(self.group.NAME or "Неизвестно"),
                                        "Такой дисциплины не было найдено в INDEX_TO_SUBJECT_DICT"
                                    )
                        
                        self.PAIRS_INT = copy.deepcopy(int_pairs_arr) # также, перепишем то, как будет выглядеть наше int'овое представление пар

                        return succeed_execution, self.PAIRS, self.PAIRS_INT # в результате возвращаем какой получился массив пар
                    
                    # ******************** [ Инициализировать дни ] ********************
                    def init_time_slots(self, debug = False):
                        for time_slot_id in range(self.timeTable.PARAM_MAX_PAIRS_AMOUNT):
                            if debug: print("\t\tAdding time slot", time_slot_id, "(", self.timeTable.INDEX_TO_TIME_SLOTS_DICT[time_slot_id + 1] ,") to day:", self.NAME, "...")
                            self.PAIRS.append(self.timeTable.CONST_NEGATIVE_PAIR_INT)
                    
                    # *************** [ Получить количество пар в дне ] ***************
                    def get_pairs_amount_per_day(self):
                        slots_counter = Counter(self.PAIRS) # подсчитаем количество имеющихся данных
                        empty_slots_amount = slots_counter[self.timeTable.CONST_NEGATIVE_PAIR_INT] # узнаем сколько из них: отсутствие пар # no_pairs_slots_amount
                        pairs_amount = len(self.PAIRS) - empty_slots_amount # отняв у общего количества пар количество пустых слотов, получим количество пар в этом дне
                        return pairs_amount
                    
                    # ***** [ Получить количество конкретной пары в текущем дне ] *****
                    def get_pair_amount_by_link(self, link, debug = False):
                        slots_counter = Counter(self.PAIRS) # подсчитаем количество имеющихся данных
                        pair_amount = 0
                        try:
                            pair_amount = slots_counter[link]
                            # if debug: print("Pair", link.SUBJECT_NAME, "amount is", pair_amount)
                        except:
                            if debug: print("Pair", link.SUBJECT_NAME, "not found in", self.NAME)

                        return pair_amount
                    
                    # *** [ Проверить, пара это или нет, либо по указанному time-slot ] ****
                    def isPair(self, pair = None, timeSlot_idx = -1):
                        if pair:
                            if isinstance(pair, self.Pair): # self.PAIRS[idx] is trully Pair
                                return True
                        elif timeSlot_idx != -1:
                            try:
                                if isinstance(self.PAIRS[idx], self.Pair): # self.PAIRS[idx] is trully Pair
                                    return True
                            except KeyError as err:
                                self.error("Возникла ошибка при проверки time-slot'а на занятость парой. Возможно был указан некорректный ID time-slot'а.")
                                self.error("Вот что выводит ошибка:", err)
                        
                        return False
                    
                    # *** [ Визуализация дня в виде массива числового эквивалента ] ****
                    def visualize(self, printResult = True, returnResult = False):
                        day_arr_result = []
                        
                        day_arr_result = self.transform_pairs_to_int_arr()
                        
                        if printResult:
                            print(day_arr_result)

                        if returnResult:
                            return day_arr_result

                    # !!!!!!!!!!!!!!!!!!!! [ Жесткие ограничение ] !!!!!!!!!!!!!!!!!!!!
                    def CHECK_STRICT_CONSTRAINT_pairs_in_the_same_corpuses(self):
                        '''
                            Запрещены переезды между корпусами в течение дня
                        '''
                        corpuses_arr = []
                        for pair in self.PAIRS:
                            if self.isPair(pair): # проверим, если это пара, а не другое значение
                                corpuses_arr.append(
                                    pair.CORPUS # добавляем название корпуса пары
                                ) #corpus for corpus in pair.CORPUS )

                        corpuses_arr = list(set(corpuses_arr))
                        corpuses_changed = 0 # UPD для статистики
                        if len(corpuses_arr) > 1: # если среди уникальных значений несколько значений
                            # return False # значит есть разные корпуса и это ограничение нарушается
                            
                            # UPD для статистики: сколько было смен корпусов в течение дня:
                            remembered = None
                            for curr_corpus in corpuses_arr:
                                if remembered is None:
                                    remembered = curr_corpus
                                    continue
                                
                                if remembered != curr_corpus:
                                    corpuses_changed += 1
                                    remembered = curr_corpus
                            
                            return False, corpuses_changed
                        
                        return True, corpuses_changed
                    
                    # ~~~~~~~~~~~~~~~~~~~~ [ Мягкие ограничение ] ~~~~~~~~~~~~~~~~~~~~
                    def CHECK_WEAK_CONSTRAINT_pairs_amount_per_day(self, param_limit = 3, except_days=[]):
                        '''
                            Количество пар в день у любой группы не должно превышать заданного количества
                        '''
                        if self.NAME in except_days: # если данный день исключение (например, в этот день весь день майнор)
                            return True # то сразу вернем True, майнор в Вышкинском расписании пишется как весь день идущий

                        pairs_amount = self.get_pairs_amount_per_day()

                        if pairs_amount > param_limit: # если количество пар больше заданного лимита, значит ограничение нарушено
                            return False, (pairs_amount-param_limit) # not satisfy
                        
                        return True, 0 # satisfy

                      
                    # ----------------------------------------------------------------------
                    def CHECK_WEAK_CONSTRAINT_windows_amount_per_day(self, param_limit = 0): # freq = frequency
                        '''
                            Желательно, чтобы количество окон сводилось к минимуму (Если param_limit указан как 1, то это значит что мы: проверим, нет ли больше одного окна в день)
                        '''
                        self.transform_pairs_to_int_arr()
                        
                        window_counter = 0 # счетчик окон
                        EMPTY_IDX = 0 # index ячейки в массиве дня, где нет пары
                        RESULT = True # результат проверки на ограничение


                        while EMPTY_IDX >= 0:
                            # print("ROOT:\n", pairs_arr)
                            try:
                                EMPTY_IDX = self.PAIRS_INT.index(self.timeTable.CONST_NEGATIVE_PAIR_INT)
                            except:
                                EMPTY_IDX = -1
                            
                            if EMPTY_IDX is -1:
                                break

                            left_arr = self.PAIRS_INT[0:EMPTY_IDX + 1]
                            right_arr = self.PAIRS_INT[EMPTY_IDX + 1:len(self.PAIRS_INT)]

                            if (left_arr and max(left_arr) > 0) and (right_arr and max(right_arr) > 0):
                                window_counter += 1
                                
                            elif (window_counter > 0 and ( (left_arr and max(left_arr) > 0) or (self.timeTable.CONST_NEGATIVE_PAIR_INT in left_arr) ) ) and (right_arr and max(right_arr) > 0): # and right_arr:
                                window_counter += 1
                                
                            elif (window_counter > 0 and ( (self.timeTable.CONST_NEGATIVE_PAIR_INT in left_arr) and (right_arr and max(right_arr) > 0) ) ): # для ситуации типа [0], [1] (это еще можно засчитать за окно), а вот [1], [0] - нет
                                win_counter += 1
                                
                            self.PAIRS_INT = right_arr

                        if window_counter > param_limit: # 0:
                            RESULT = False

                        return RESULT, window_counter # True, если нет никаких с этим проблем
                    
                    # ----------------------------------------------------------------------
                    def CHECK_WEAK_CONSTRAINT_preferable_max_pairs_per_day(self, max_pairs = -1):
                        '''
                            Желательно, чтобы количество пар в день у любой учебной группы не превышало заданного значения
                        '''
                        if max_pairs == -1:
                            max_pairs = self.timeTable.PARAM_PREFER_MAX_PAIRS_PER_DAY

                        real_pairs_amount = self.get_pairs_amount_per_day()
                        diff = max_pairs - real_pairs_amount
                        
                        if diff >= 0:
                            return True, 0
                        else:
                            return False, abs(diff)
                    
                    # ----------------------------------------------------------------------
                    def CHECK_WEAK_CONSTRAINT_preferable_min_pairs_per_day(self, min_pairs = -1):
                        '''
                            Желательно, чтобы количество пар в день у любой учебной группы было больше или равно заданного значения или же равно нулю.
                        '''
                        if min_pairs == -1:
                            min_pairs = self.timeTable.PARAM_PREFER_MIN_PAIRS_PER_DAY

                        real_pairs_amount = self.get_pairs_amount_per_day()
                        diff = min_pairs - real_pairs_amount
                        
                        if real_pairs_amount >= min_pairs or real_pairs_amount == 0:
                            return True, 0
                        else:
                            return False, abs(diff)
                    
                    # ----------------------------------------------------------------------
                    def CHECK_WEAK_CONSTRAINT_changing_auditoriums_per_day_amount(self):
                        '''
                            Желательно, чтобы количество различных предметов в день у любой учебной группы превышало или было равно количеству аудиторий, в которых проводятся эти самые занятия в течение дня
                            Если количество переходов = количеству занятий, то штраф 1
                        '''
                        all_pairs_amount = self.get_pairs_amount_per_day()
                        all_rooms = []
                        for pair in self.PAIRS:
                            if self.isPair(pair):
                                for room in pair.ROOM:
                                    all_rooms.append(room)
                        
                        unique_rooms = list(set(all_rooms))
                        diff = all_pairs_amount - len(unique_rooms)

                        if diff == 0:
                            return False, diff
                        
                        return True, diff

                    # ----------------------------------------------------------------------
                    def CHECK_WEAK_CONSTRAINT_less_preferable_pair_timeslots(self):
                        '''
                            Желательно, чтобы в течение дня пары у любой учебной группы располагались в заданном заранее промежутке
                            Первая и последние пары менее желательны, чем остальные (т.е там, где нули, они менее желательны: [0, 1, 1, 1, 1, 0]). За каждую пару в этом промежутке штраф = 1
                        '''
                        less_preferable_pairs_amount = 0
                        if self.isPair(self.PAIRS[0]): # если у нас первой парой что-то стоит (какая-то пара)
                            less_preferable_pairs_amount += 1
                        
                        if self.isPair(self.PAIRS[-1]): # если у нас последней парой что-то стоит (какая-то пара)
                            less_preferable_pairs_amount += 1

                        if less_preferable_pairs_amount > 0: # если в таких timeslot'ах все-таки нашлись пары, значит не удовлетворяет условиям
                            return False, less_preferable_pairs_amount

                        return True, less_preferable_pairs_amount

                    # ==================== [ Сущность пары ] ====================

#########################################################################################################################################################################################
#########################################################################################################################################################################################
#########################################################################################################################################################################################
                    # ===================== [ Сущность пары ] ======================
                    class Pair(Utils):
                    # __________ [ Pair ] __________
                        # ******************** [ Инициализация ] ********************
                        def __init__(self, timeTable, group, week, day, university,
                            name = 'Дисциплина без названия',
                            tutors = [], # tutor = '-',
                            rooms = [], # room = '-',
                            pairType = '',
                            extra = '',
                            corpus = '',
                            groups = [] # group = ''
                        ):
                            self.university = university
                            self.timeTable = timeTable
                            self.group = group
                            self.week = week
                            self.day = day
                            #-------------------------
                            
                            self.SUBJECT_NAME = name
                            self.TEACHER = tutors
                            self.ROOM = rooms
                            self.TYPE = pairType
                            self.EXTRA_INFO = extra
                            self.CORPUS = corpus

                            self.GROUPS_LIST = groups # may be its bettter to keep GROUP_ID
                            # self.OTHER_GROUPS = [] # когда пара проводится и для других групп

                        def visualize(self):
                            print(self.SUBJECT_NAME, "|", self.TYPE, "| Rooms:", self.ROOM, "|", self.CORPUS, "|", self.TEACHER, "|", self.GROUPS_LIST, "|", self.EXTRA_INFO)

                        # !!!!!!!!!!!!!!!!!!!! [ Жесткие ограничения ] !!!!!!!!!!!!!!!!!!!!
                        def CHECK_WEAK_CONSTRAINT_auditorium_satisfies_pair_type(self, debug = True):
                            '''
                                Тип аудитории должен подходить для типа занятия
                            '''
                            matching = {
                                "Лекционная": "Лекция",
                                "Семинарская": "Семинар",
                                "Компьютерная": "-",
                                "Неизвестно": "Неизвестный тип"
                            }
                            
                            match_amount = 0
                            mismatch_amount = 0


                            for room in self.ROOM:
                                if room in self.timeTable.NEGATIVE_VALUES:
                                    # print("\tSKIPPED ROOM:", room)
                                    continue
                                
                                # print("ROOM:", room)

                                room_type = self.university.DB_AUDITORIUM[self.CORPUS][room][self.timeTable.CONST_LTYPE]
                                if matching[room_type] != self.TYPE:
                                    mismatch_amount += 1
                                else:
                                    match_amount += 1

                            if mismatch_amount > 0:
                                return False, mismatch_amount
                            else:
                                return True, mismatch_amount

                        # ~~~~~~~~~~~~~~~~~~~~ [ Мягкие ограничение ] ~~~~~~~~~~~~~~~~~~~~
                        def CHECK_WEAK_CONSTRAINT_pairs_chaining_satisfies(self):
                            '''
                                Желательно, чтобы одно занятие располагалось после другого, если такая связь указана для двух занятий
                            '''
                            target = None
                            LECTURE = self.timeTable.CONST_LECTURE
                            PRACTICE = self.timeTable.CONST_PRACTICE
                            try:
                                target = self.timeTable.PAIRS_CHAIN[self.group.NAME][self.SUBJECT_NAME]
                                # если длина цепочки пар > 1 (т.е есть лекция и практика у данного предмета (и для лекции информация записана и для практики))
                                if len(target) > 1:
                                    # если недели между лекцией и практикой совпадают:
                                    if target[LECTURE].week == target[PRACTICE].week:
                                        # и если еще и дни  между лекцией и практикой совпадают:
                                        if target[LECTURE].day == target[PRACTICE].day:

                                            lecture_timeSlotIDx = target[LECTURE].day.PAIRS.index(target[LECTURE]) # узнаем индекс пары лекции среди всех пар этого дня
                                            practice_timeSlotIDx = target[PRACTICE].day.PAIRS.index(target[PRACTICE]) # узнаем индекс пары практики среди всех пар этого дня
                                            diff = abs(lecture_timeSlotIDx - practice_timeSlotIDx) # получим абсолютное значение разницы их позиций и выясним, есть ли какие-либо пары между ними, или они стоят друг за другом
                                            
                                            if diff == 1:
                                                return True, diff - 1 # удовлетворяет условию
                                            
                                            # если разница больше чем в одну пару
                                            elif diff > 2:
                                                return False, diff - 1 # НЕ удовлетворяет условию и дополнительно возвращает сколько пар между нашими target'ами
                            except: # такой связи не установлено
                                pass
                            return False, -100

                    # ==================== [ /Сущность пары ] ======================

                # ================== [ /День недели расписания ] ===================
        
            # ====================== [ /Неделя расписания ] ========================
        
        # =========== [ /Группа, для которой предназначено расписание ] ============
        
        # ======== [ Функция для назначения штрафов и фитнесса в статистику группы ] ========
        def set_fine_stats(self, group, constraint_name, fine_size, fine_amount_increment_by = 1, fine_violation_accumulation_by = -1, whole_general_fitness_value_by = -1):
            # __________________________________________________________________________________________________________________________________________________________________
            # накопленные величины                  - если не задано кастомное значение параметра fine_violation_accumulation_by, то по умолчанию:
            # (fine_violation_accumulation_by)        это аккумуляция (накапливание) значений: (величины нарушения ограничения, помноженного на self.FINES[constraint_name],
            #                                         т.е стандартный fine_size)
            # __________________________________________________________________________________________________________________________________________________________________
            # счетчик накопленных величин штрафов   - если не задано кастомное значение параметра whole_general_fitness_value_by, то по умолчанию:
            # (whole_general_fitness_value_by)        копится общее количество величин штрафов за все вместе взятые ограничения, а не только текущего ограничения
            #                                         То есть по этому полю в дальнейшем можно узнать общий фитнесс нарушений на целую группу за всю сессию проверки ограничений
            # __________________________________________________________________________________________________________________________________________________________________

            # constraint_name - наименование ограничения
            # fine_size - величина штрафа
            # fine_amount_increment_by - на сколько увеличить счетчик нарушения данного ограничения (по умолчанию на 1 еденицу)
            # fine_violation_accumulation_by - на сколько увеличить счетчик накопленных величин нарушения данного ограничения (по умолчанию не задан (-1), а значит берем значение fine_size)
            # whole_general_fitness_value_by - на сколько увеличить счетчик накопленных величин штрафов за нарушение данного ограничения (по умолчанию не задан (-1), а значит берем значение fine_size)

            # ****************************** [ASSIGNING FINES] ******************************
            if fine_violation_accumulation_by == -1: fine_violation_accumulation_by = fine_size
            if whole_general_fitness_value_by == -1: whole_general_fitness_value_by = fine_size
            
            group.STATISTICS[constraint_name][0] += fine_amount_increment_by # инкрементируем количество нарушенных ограничений этого типа
            group.STATISTICS[constraint_name][1] += fine_violation_accumulation_by # инкрементируем величину штрафов за ограничение этого типа
            group.STATISTICS["WHOLE_GENERAL_FITNESS"] += whole_general_fitness_value_by # величина штрафа за все время (за все ограничения)
            # ***************************** [/ASSIGNING FINES] ******************************

        # ======== [ Функция для подсчета фитнесса ] ========
        # На данный момент задействованно 13 ограничений
        def calc_fitness(self, particular_groups = [], debug = False):
            calculated_groups_links = [] # массив с сылками на группы, для которых был посчитан фитнесс и у которых можно потом посмотреть .STATISTICS
            
            for group_name in particular_groups or self.GROUPS:

                # LOCAL_FITNESS_SUM = 0
                current_group = self.select_group(group_name, reDefine = False)
                current_group.reset_stats()

                # а у этих ограничений фитнесс как-то прибавляется?
                # ------------------------- [ Group ] -------------------------
                # ====== * Strict Constraints
                # 9. Количество пар конкретного предмета в расписании должно соответствовать их количеству в учебном плане
                pairs_amount_per_eduPlan_satisfied, pairs_amount_per_eduPlan_fitness = current_group.LAUNCH_CHECK_STRICT_CONSTRAINT_subject_pairs_amount_per_eduPlan()
                if pairs_amount_per_eduPlan_satisfied is False: # если ограничение нарушено
                        self.set_fine_stats(current_group, "subject_pairs_amount_per_eduPlan", fine_size = pairs_amount_per_eduPlan_fitness) # назначаем штраф за нарушение данного ограничения

                # ====== * Weak Constraints
                # 14. Желательно, чтобы пары одного и того же предмета проводились в одни и те же дни и временные интервалы каждую неделю в одних и тех же аудиториях
                module_stats = current_group.CHECK_WEAK_CONSTRAINT_the_same_pairs_position_by_week()
                
                # ------------------------- [ All the weeks ] -------------------------
                # ====== * Weak Constraint
                for week in current_group.WEEKS:

                    # ====== * Strict Constraints

                    # 2. Количество пар в неделю у любой группы не должно превышать заданного количества
                    pairs_amount_per_week_satisfied, pairs_amount_per_week_fitness_value = week.LAUNCH_CHECK_WEAK_CONSTRAINT_pairs_amount_per_week()
                    if pairs_amount_per_week_satisfied is False: # если ограничение нарушено
                        self.set_fine_stats(current_group, "pairs_amount_per_week", fine_size = pairs_amount_per_week_fitness_value) # назначаем штраф за нарушение данного ограничения
                    
                    # ====== * Weak Constraints

                    # 16. Желательно, чтобы количество пар у любой учебной группы в течение субботы сводилось к минимуму
                    saturday_pairs_satisfied, saturday_pairs_amount = week.CHECK_WEAK_CONSTRAINT_saturday_pairs_amount()
                    if saturday_pairs_satisfied is False: # если ограничение нарушено
                        self.set_fine_stats(current_group, "saturday_pairs_amount", fine_size = (saturday_pairs_amount * self.FINES["saturday_pairs_amount"])) # назначаем штраф за нарушение данного ограничения

                    # ------------------------- [ All the days ] -------------------------
                    for day in week.DAYS:

    
                        # 12. Желательно, чтобы количество пар в день у любой учебной группы не превышало заданного значения
                        max_pairs_satisfied, max_pairs_diff = day.CHECK_WEAK_CONSTRAINT_preferable_max_pairs_per_day()
                        if max_pairs_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "preferable_max_pairs_per_day", fine_size = max_pairs_diff * self.FINES["preferable_max_pairs_per_day"]) # назначаем штраф за нарушение данного ограничения

    
                        # 13. Желательно, чтобы количество пар в день у любой учебной группы было больше или равно заданного значения или же равно нулю
                        min_max_pairs_satisfied, min_max_pairs_diff = day.CHECK_WEAK_CONSTRAINT_preferable_min_pairs_per_day()
                        if min_max_pairs_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "preferable_min_pairs_per_day", fine_size = min_max_pairs_diff * self.FINES["preferable_min_pairs_per_day"]) # назначаем штраф за нарушение данного ограничения
                        
    
                        # 15. Желательно, чтобы количество различных предметов в день у любой учебной группы превышало или было равно количеству аудиторий, в которых проводятся эти самые занятия в течение дня
                        changing_auditoriums_per_day_satisfied, changing_auditoriums_per_day_amount = day.CHECK_WEAK_CONSTRAINT_changing_auditoriums_per_day_amount()
                        if changing_auditoriums_per_day_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "changing_auditoriums_per_day_amount", fine_size = changing_auditoriums_per_day_amount * self.FINES["changing_auditoriums_per_day_amount"]) # назначаем штраф за нарушение данного ограничения

    
                        # 17. Желательно, чтобы в течение дня пары у любой учебной группы располагались в заданном заранее промежутке
                        less_preferable_pair_satisfied, less_preferable_pair_amount = day.CHECK_WEAK_CONSTRAINT_less_preferable_pair_timeslots()
                        if less_preferable_pair_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "less_preferable_pair_timeslots", fine_size = less_preferable_pair_amount * self.FINES["less_preferable_pair_timeslots"]) # назначаем штраф за нарушение данного ограничения

                        # 1. Количество пар в день у любой группы не должно превышать заданного количества:
                        pairs_amount_per_day_satisfied, pairs_amount_per_day_violation = day.CHECK_WEAK_CONSTRAINT_pairs_amount_per_day(
                            param_limit = 3
                        )
                        if pairs_amount_per_day_satisfied is False: # если ограничение нарушено
                            continue
                            #self.set_fine_stats(current_group, "pairs_amount_per_day", fine_size = pairs_amount_per_day_violation * self.FINES["pairs_amount_per_day"]) # назначаем штраф за нарушение данного ограничения

    
                        # 11. Желательно, чтобы количество окон сводилось к минимуму
                        windows_amount_per_day_satisfied, windows_amount = day.CHECK_WEAK_CONSTRAINT_windows_amount_per_day() # (1) # Проверим, нет ли больше одного окна в день
                        if windows_amount_per_day_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "windows_amount_per_day", fine_size = windows_amount * self.FINES["windows_amount_per_day"]) # назначаем штраф за нарушение данного ограничения
                        
    
                        # 3. Запрещены переезды между корпусами в течение дня
                        was_corpus_changed_satisfied, corpus_changed_amount = day.CHECK_STRICT_CONSTRAINT_pairs_in_the_same_corpuses()
                        if was_corpus_changed_satisfied is False: # если ограничение нарушено
                            self.set_fine_stats(current_group, "pairs_in_the_same_corpuses", fine_size = corpus_changed_amount * self.FINES["pairs_in_the_same_corpuses"]) # назначаем штраф за нарушение данного ограничения
                        
    

                        # ------------------------- [ All the pairs ] -------------------------
                        tmp_pairs_checked = []

                        for pair in day.PAIRS:
                            if day.isPair(pair): # забавная иерархия и проверка получается (если бы использовали: pair.day.isPair(pair), но т.к pair может выступать числом, то до .day дотянутся не получится...) :) (т.к проверка на то: является ли пара действительно парой (экземпляром класса Pair) реизована в классе Day, то приходится сперва до него тянуться, а потом уже и метод вызывать, передавая нужный нам объект)
                                # ====== * Strict Constraint
                                # 5. Тип аудитории должен подходить для типа занятия
                                pair_types_satisfies, pair_types_mismatches = pair.CHECK_WEAK_CONSTRAINT_auditorium_satisfies_pair_type()
                                if pair_types_satisfies is False: # если ограничение нарушено
                                    self.set_fine_stats(current_group, "auditorium_satisfies_pair_type", fine_size = pair_types_mismatches * self.FINES["auditorium_satisfies_pair_type"]) # назначаем штраф за нарушение данного ограничения

                                # ====== * Weak Constraint
                                # 10. Желательно, чтобы одно занятие располагалось после другого, если такая связь указана для двух занятий
                                pair_chain_satisfies, pair_chain_between = pair.CHECK_WEAK_CONSTRAINT_pairs_chaining_satisfies()
                                if pair_chain_satisfies is False and pair_chain_between != -100: # если ограничение нарушено (и тут же проверяем частный случай на значение -100, который сообщает о том, что такой связи (чтобы одно занятие располагалось после другого) не установлено)
                                    self.set_fine_stats(current_group, "pairs_chaining_satisfies", fine_size = pair_chain_between * self.FINES["pairs_chaining_satisfies"]) # назначаем штраф за нарушение данного ограничения

                                if pair.SUBJECT_NAME not in tmp_pairs_checked:
                                    # print("CHECKING week_max_particular_subject_pairs_amount")
                                    week_max_particular_subject_pairs_amount_satisfied, violated_week_max_particular_subject_pairs_amount = pair.week.CHECK_WEAK_CONSTRAINT_week_max_particular_subject_pairs_amount(pair.SUBJECT_NAME)
                                    tmp_pairs_checked.append(pair.SUBJECT_NAME)
                                    
                                    # назначаем штрафы:
                                    if week_max_particular_subject_pairs_amount_satisfied is False and violated_week_max_particular_subject_pairs_amount > 0: # если ограничение нарушено
                                        # print("\tweek_max_particular_subject_pairs_amount was violated")
                                        self.set_fine_stats(current_group, "week_max_particular_subject_pairs_amount", fine_size = violated_week_max_particular_subject_pairs_amount * self.FINES["week_max_particular_subject_pairs_amount"]) # назначаем штраф за нарушение данного ограничения


                # раз мы дошли до этого места, значит выше не было никаких ошибок и fitness для текущей группы посчитался, можно добавлять в массив ссылку на группу для которой только что был посчитан fitness
                calculated_groups_links.append(current_group)

                    # break # пока один раз посчитаем, т.к остальные недели дублируются
            if debug: print("calc_fitness finished...")
            return True, calculated_groups_links
    # =============================== [ /Расписание ] ==============================

---
## **[INITIAL]** Функция получения набора из начальных решений посредством случайного поиска **для каждой указанной группы**

In [ ]:
# Возвращает по итогу для каждой группы массив с сылками на эту созданную группу с проинициализированной для нее random расписанием (initial solution'ом)
def init_and_get_initial_solutions(particular_groups = ["16БИ-2"], solutions_amount_per_group = 5, visualize_result = True, whole_stats = False,  debug = False):
    groupsBy_result = {} # initial_solution для каждой указанной группы (по факту массив ссылок на все те группы, для которых была проинициализирована структура и для которых посчитался fitness)

    for groupName in particular_groups:
        text_prefix = ''
        if len(particular_groups) > 1:
            print("############################################################ Рассматриваемая группа:", groupName, "############################################################")
            text_prefix = '\t'

        groupsBy_result.update({ groupName: [] }) # инициализируем массив с initial solutions для группы
        # т.к нам нужна новая ссылка на объект для каждого initial solution'а, то помещаем всю структуру ниже под цикл solution_number'ов
        for solution_number in range(solutions_amount_per_group): # сгенерируем необходимое количество initial solution'ов
            univer = University( auditorium_db_file_path = './auditorium_db.json')
            timeTable = univer.ModuleTimetable(univer)
            timeTable.prepare_data(
                need_particular_info = True,
                particular_group = particular_groups,
                debug = debug
            )
            timeTable.init__groups_links() # инициализация ссылок на группы
            
            curr_group_link = timeTable.select_group(groupName) # выбор группы на рассмотрение
            curr_group_link.init__empty_all(debug = False) # инициализация всех необходимых сущностей
            if debug: curr_group_link.visualize()

            

            # curr_group_link.random_timetable() # генерация initial solution / random soluton для группы
            # timeTable.calc_fitness( particular_groups = [groupName] ) # т.к мы и так проходимся по всем timeTable.GROUPS, то посчитаем фитнесс конкретно для текущей группы и для этой группы уже фитнесс считай посчитан

            random_search_solution_attempts = -1
            rerun = True
            while rerun == True: # будем пробовать генерировать начальное решение до тех пор, пока имеется наличие жестких нарушений
                _, random_search_solution_attempts = curr_group_link.launch_random_search(total_attempts = 10000) # получим начальное решение с помощью случайно составленного расписания
                rerun = curr_group_link.has_hard_constraints()

            print("\n\n\ncurr_group_link.PAIR_LINK:", curr_group_link.PAIR_LINK, "\n\n\n")
            
            groupsBy_result.update({ # обновим список initial solution'ов для текущей группы 
                groupName: [ *groupsBy_result[groupName], curr_group_link ]
            })

            if visualize_result:
                print(text_prefix + "------------------------------ [ Начальное решение №" + str(solution_number + 1) + " для группы:", groupName, "] ------------------------------")
                curr_group_link.visualize()
                print(text_prefix + "________________________________________________________________________________________________________________")
                print(text_prefix + "* SOLUTION FITNESS:", curr_group_link.STATISTICS["WHOLE_GENERAL_FITNESS"])
                print(text_prefix + "* Для генерации решения понадобилось попыток:", random_search_solution_attempts)
                
                if whole_stats:
                    print(text_prefix + "____________________ Статистика решения: ____________________")
                    for stat in curr_group_link.STATISTICS:
                        print(text_prefix + "\t", stat, ":", curr_group_link.STATISTICS[stat])
                print(text_prefix + "----------------------------------------------------------------------------------------------------------------\n\n\n")
    return groupsBy_result

---
## **[LOCAL SEARCH 1]** Функция Локального поиска №1

In [ ]:
def local_search_v1(initial_solution_group_link, max_failure_iterations = 1000, max_repeated_the_same_suggestions = -1, debug = False):
    # initial_solution_group_link - ссылка на группу, у которой уже было проинициализированно начальное решение
    # max_failure_iterations - максимальное количество неудачных итераций (попыток) улучшить расписание
    # max_repeated_the_same_suggestions - максимальное количество предложенных уже рассмотренных алгоритмом решений (ходов)
        # Если параметр max_repeated_the_same_suggestions не задан (равен -1), то ниже по умолчанию будет проинициализировано его значение равное:
            # максимальному количеству возможных ходов (всех ячеек (временных слотов)) для рассмотрения в расписании путем переумножения следующих параметров:
                # PARAM_MAX_PAIRS_AMOUNT (максимальное количество пар в день в расписании (6)) * PARAM_STUDY_DAYS_PER_WEEK_AMOUNT (максимальное количество учебных дней в неделю в расписании (6)) * PARAM_MODULE_WEEKS_AMOUNT (максимальное количество недель в модуле в расписании (11)) = 432

    if max_repeated_the_same_suggestions == -1:
        group_tt = initial_solution_group_link.timeTable # получим расписание группы
        group_tt_settings = [group_tt.PARAM_MAX_PAIRS_AMOUNT, group_tt.PARAM_STUDY_DAYS_PER_WEEK_AMOUNT, group_tt.PARAM_MODULE_WEEKS_AMOUNT] # сформируем массив из необходимых параметров для получения значения максимального количества возможных ячеек (time-slot'ов в расписании)
        max_repeated_the_same_suggestions = group_tt_settings[0] * group_tt_settings[1] * group_tt_settings[2] # перемножим параметры расписания и получим максимальное количество возможных ячеек (time-slot'ов в расписании)

    relevant_fragment = copy.deepcopy(initial_solution_group_link) # создаем слепок текущего решения
    draft_fragment = copy.deepcopy(relevant_fragment) # создаем слепок текущего решения для дальнейших экспериментов
    # print("Проверяем, что: relevant_fragment (",id(relevant_fragment),") - не тот же самый, что и: draft_fragment (",id(draft_fragment),"):", relevant_fragment is not draft_fragment)
    SUBJECT_FAILURE_COUNTER = 0 # переменная-счетчик для хранения сколько неудачных попыток перемещения конкретной пары подряд уже было совершено (например 286 раз предмет не удалось переместить никуда - это предел)
    WEEK_FAILURE_COUNTER = 0 # переменная-счетчик для хранения сколько таких неудачных попыток перемещения конкретной пары подряд уже было совершено (например сколько таких предметов, которых не удалось переместить 286 раз. И таких "неудалось переместить предметов" тоже можно только 286 раз рассмотреть)

    print("[BEFORE START] relevant_fragment_fitness:", relevant_fragment.STATISTICS['WHOLE_GENERAL_FITNESS'])
    # print("[BEFORE START] draft_fragment_fitness:", draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS'])

    # while SUBJECT_FAILURE_COUNTER < max_failure_iterations:
    # Последовательно идем по всем неделям, дням и парам:
    while WEEK_FAILURE_COUNTER < max_failure_iterations:
        print("WEEK_FAILURE_COUNTER:", WEEK_FAILURE_COUNTER)
        for weekID in range(len(draft_fragment.WEEKS)):
            if debug:
                print("\n===================================================================================================================================================================================")
                print("[", weekID, "] ", end="")
                draft_fragment.WEEKS[weekID].visualize()
                print("\t__________________________________________________________________________________________________________________________")

            for dayID in range(len(draft_fragment.WEEKS[weekID].DAYS)):
                if debug:
                    print("\tCURRENT DAY IS №", dayID, ": ", end="")
                    draft_fragment.WEEKS[weekID].DAYS[dayID].visualize()
                    print("\t\t_______________________________________________________")

                for pairID in range(len(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS)):
                    if debug: print("\t\tCURRENT PAIR №", pairID, ": ", end="")

                    if draft_fragment.WEEKS[weekID].DAYS[dayID].isPair(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID]): # Натыкаемся на непустую ячейку с каким-то предметом (если текущая пара (ячейка) действительно пара)
                        if debug: draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID].visualize()

                        
                        fitness_changed_better = False
                        SUBJECT_FAILURE_COUNTER = 0 # сбрасываем
                        REMEMBERED_W_D_P_SEQUENCE = [] # массив с запомненными вариантами тайм-слотов
                        REPEATED_THE_SAME_SUGGESTION_COUNTER = 0 # счетчик количества уже ранее рассмотренных решений
                        while fitness_changed_better is False and SUBJECT_FAILURE_COUNTER < max_failure_iterations: # 
                            # попытаемся выбрать рандомно свободную ячейку для дальнейшего перемещения найденной пары:
                            if debug: print("\n\t\t\t>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                            if debug: print("\t\t\tSUBJECT_FAILURE_COUNTER:", SUBJECT_FAILURE_COUNTER, "/", max_failure_iterations, "|", "fitness_changed_better:", fitness_changed_better)
                            if debug: print("\t\t\tREPEATED_THE_SAME_SUGGESTION_COUNTER:", REPEATED_THE_SAME_SUGGESTION_COUNTER, "/", max_repeated_the_same_suggestions, "|",  "len(REMEMBERED_W_D_P_SEQUENCE):", len(REMEMBERED_W_D_P_SEQUENCE))

                            try:
                                
                                prev_draft_fitness = draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS']
                                if debug: print("\t\t\t\tprev_draft_fitness:", prev_draft_fitness)

                                # ---------------------------------------- MOVEMENT ----------------------------------------
                                if debug: print("\t\t\t\t\tv----Попытка переместится с : [weekID =", weekID, ", dayID =", dayID, ", pairID =", pairID, "]")
                                if debug: print("\t\t\t\t\t|", draft_fragment.WEEKS[weekID].visualize(printResult = False, returnResult = True),"\n\t\t\t\t\t| А именно с:", draft_fragment.WEEKS[weekID].DAYS[dayID].visualize(printResult = False, returnResult = True))
                                if debug: # с...
                                    print("\t\t\t\t\t>-------------", end="")
                                    for day_gap in range(pairID + 1):
                                        print("--", end="")
                                    print("^")
                                
                                # максимальное количество попыток для нахождения свободной ячейки будет равносильно количеству всех возможных ячеек (помноженные длины массивов каждых сущностей друг на друга)
                                freeWeekID, freeDayID, freeTimeSlotID = draft_fragment.get_random_free_cell(maximum_attempts = (len(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS) * len(draft_fragment.WEEKS[weekID].DAYS) * len(draft_fragment.WEEKS)) ) # ищем свободную ячейку


                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t|Попытка переместится в : [freeWeekID =", freeWeekID, ", freeDayID =", freeDayID, ", freeTimeSlotID =", freeTimeSlotID, "]")
                                if debug: print("\t\t\t\t\t|", draft_fragment.WEEKS[freeWeekID].visualize(printResult = False, returnResult = True),"\n\t\t\t\t\t| А именно в:", draft_fragment.WEEKS[freeWeekID].DAYS[freeDayID].visualize(printResult = False, returnResult = True))
                                if debug: # в...
                                    print("\t\t\t\t\t>------------", end="")
                                    for day_gap in range(freeTimeSlotID + 1):
                                        # if day_gap == 0: print("", end="")
                                        # else:
                                            # print("--", end="")
                                            print("--", end="")
                                    for addition in range(day_gap):
                                        print("-", end="")
                                    print("^")

                                
                                if [freeWeekID, freeDayID, freeTimeSlotID] in REMEMBERED_W_D_P_SEQUENCE: # если такая выборка уже происходила
                                    if debug: print("\t\t\t\t\tТакой [freeWeekID =", freeWeekID, ", freeDayID =", freeDayID, ", freeTimeSlotID =", freeTimeSlotID, "] уже рассматривался")
                                    REPEATED_THE_SAME_SUGGESTION_COUNTER += 1
                                    
                                    if REPEATED_THE_SAME_SUGGESTION_COUNTER >= max_repeated_the_same_suggestions: # if len(REMEMBERED_W_D_P_SEQUENCE) >= max_repeated_the_same_suggestions:
                                        if debug: print("\t\t\t\t\t\tВыявлено, что ни одно предложенное решение не улучшает результат. (Все возможные решения уже были рассмотренны)\n\t\t\t\t\t\tПрекращаем цикл while...")
                                        break
                                    continue # заставляем итерацию while-цикла повторится 


                                draft_fragment.WEEKS[weekID].DAYS[dayID].move_pair(pairID, freeWeekID, freeDayID, freeTimeSlotID) # перемещаем текущую pairID пару на найденные свободные позиции: freeWeekID, freeDayID, freeTimeSlotID
                                
                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t|\t ------------------------------------------------------------------------------------------------------------")
                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t| Результат 'с':", draft_fragment.WEEKS[weekID].visualize(printResult = False, returnResult = True), "\n\t\t\t\t\t| А именно с:", draft_fragment.WEEKS[weekID].DAYS[dayID].visualize(printResult = False, returnResult = True))
                                if debug: # с...
                                    print("\t\t\t\t\t>-------------", end="")
                                    for day_gap in range(pairID + 1):
                                        print("--", end="")
                                    print("^")

                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t| Результат 'в':", draft_fragment.WEEKS[freeWeekID].visualize(printResult = False, returnResult = True), "\n\t\t\t\t\t| А именно в:", draft_fragment.WEEKS[freeWeekID].DAYS[freeDayID].visualize(printResult = False, returnResult = True))
                                if debug: # в...
                                    print("\t\t\t\t\t>------------", end="")
                                    for day_gap in range(freeTimeSlotID + 1):
                                        # if day_gap == 0: print("", end="")
                                        # else:
                                            # print("--", end="")
                                            print("--", end="")
                                    for addition in range(day_gap):
                                        print("-", end="")
                                    print("^")

                                # if debug: print("\t\t\t\t\t", draft_fragment.visualize())
                                # --------------------------------------- /MOVEMENT ----------------------------------------

                                draft_fragment.timeTable.calc_fitness()

                                curr_draft_fitness = draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS']
                                if debug: print("\t\t\t\tcurr_draft_fitness:", curr_draft_fitness)
                                # if debug: print("\t\t\t\tTEST OF curr_draft_fitness:", isinstance(draft_fragment.WEEKS[weekID].DAYS[dayID], draft_fragment.Week.Day))#draft_fragment.WEEKS[weekID].DAYS[dayID].group.STATISTICS['WHOLE_GENERAL_FITNESS'])

                                # Если удалось улучшить
                                if curr_draft_fitness < prev_draft_fitness:
                                    print("\tImproved fitness: from",prev_draft_fitness, "to", curr_draft_fitness)
                                    fitness_changed_better = True # сообщаем что fitness изменился в лучшую сторону
                                    relevant_fragment = copy.deepcopy(draft_fragment) # актуализируем слепок улучшенным вариантом
                                    SUBJECT_FAILURE_COUNTER = 0 # в любом случае сбрасываем счетчик накопленных ошибочных действий подряд
                                    REMEMBERED_W_D_P_SEQUENCE = [] # запоминаем что такой слот уже посмотрели

                                    # Если удалось улучшить до того, что у fitness == 0, то сразу возвращаем полученный результат
                                    if curr_draft_fitness == 0:
                                        return relevant_fragment

                                # Если удалось только лишь ухудшить
                                else:
                                    REMEMBERED_W_D_P_SEQUENCE.append( [freeWeekID, freeDayID, freeTimeSlotID] ) # запоминаем что такой слот уже посмотрели
                                    SUBJECT_FAILURE_COUNTER += 1
                                    # print("\t!!! draft_fragment WAS CHANGED FROM:", draft_fragment, "|", "WHILE draft_fragment.WEEKS[weekID].DAYS[dayID] IS:", draft_fragment.WEEKS[weekID].DAYS[dayID].group)
                                    draft_fragment = copy.deepcopy(relevant_fragment) # возвращаемся на последний релевантный улучшенный вариант
                                    # print("\t!!! draft_fragment WAS CHANGED TO  :", draft_fragment, "|", "WHILE draft_fragment.WEEKS[weekID].DAYS[dayID] IS:", draft_fragment.WEEKS[weekID].DAYS[dayID].group)

                            # если выбрать рандомно свободную ячейку не удалось:
                            except Exception as e:
                                print("\t\t\t\tВыбор ячейки не удался")
                                print(e)
                                return

                        if fitness_changed_better is False: # and SUBJECT_FAILURE_COUNTER >= max_failure_iterations: # если улучшений так и не произошло, и достигнут максимальный порог фейловых операций (закомментированно: SUBJECT_FAILURE_COUNTER >= max_failure_iterations т.к SUBJECT_FAILURE_COUNTER может так и не сравняться/превысить max_failure_iterations)
                            WEEK_FAILURE_COUNTER += 1 # то значит найден +1 предмет, который не удалось переместить ни в какую
                            print("WEEK_FAILURE_COUNTER:", WEEK_FAILURE_COUNTER)

                    else: # если текущая рассматриваемая пара (ячейка) вовсе не пара (например, 0, а не ссылка на пару)
                        if debug: print(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID])
                        continue # пропускаем итерацию

                    if debug: print("\t\t_______________________________________________________")
                if debug: print("\t------------------------------------------------------------------------------------------------------------------------------------------------------------------")

    return relevant_fragment

---
## **[LOCAL SEARCH 2]** Функция Локального поиска №2

In [ ]:
def local_search_v2(initial_solution_group_link, max_failure_iterations = 500, debug = False):
    relevant_fragment = copy.deepcopy(initial_solution_group_link) # создаем слепок текущего решения
    draft_fragment = copy.deepcopy(relevant_fragment) # создаем слепок текущего решения для дальнейших экспериментов
    # print("Проверяем, что: relevant_fragment (",id(relevant_fragment),") - не тот же самый, что и: draft_fragment (",id(draft_fragment),"):", relevant_fragment is not draft_fragment)
    SUBJECT_FAILURE_COUNTER = 0 # переменная-счетчик для хранения сколько неудачных попыток смены пар подряд уже было совершено
    WEEK_FAILURE_COUNTER = 0 # переменная-счетчик для хранения сколько таких неудачных попыток смены пар подряд уже было совершено

    print("[BEFORE START] relevant_fragment_fitness:", relevant_fragment.STATISTICS['WHOLE_GENERAL_FITNESS'])
    # print("[BEFORE START] draft_fragment_fitness:", draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS'])

    # while SUBJECT_FAILURE_COUNTER < max_failure_iterations:
    # Последовательно идем по всем неделям, дням и парам:
    while WEEK_FAILURE_COUNTER < max_failure_iterations:
        print("WEEK_FAILURE_COUNTER:", WEEK_FAILURE_COUNTER)
        for weekID in range(len(draft_fragment.WEEKS)):
            if debug:
                print("\n===================================================================================================================================================================================")
                print("[", weekID, "] ", end="")
                draft_fragment.WEEKS[weekID].visualize()
                print("\t__________________________________________________________________________________________________________________________")

            for dayID in range(len(draft_fragment.WEEKS[weekID].DAYS)):
                if debug:
                    print("\tCURRENT DAY IS №", dayID, ": ", end="")
                    draft_fragment.WEEKS[weekID].DAYS[dayID].visualize()
                    print("\t\t_______________________________________________________")

                for pairID in range(len(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS)):
                    if debug: print("\t\tCURRENT PAIR №", pairID, ": ", end="")

                    if draft_fragment.WEEKS[weekID].DAYS[dayID].isPair(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID]): # Натыкаемся на непустую ячейку с каким-то предметом (если текущая пара (ячейка) действительно пара)
                        if debug: draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID].visualize()

                        
                        fitness_changed_better = False
                        SUBJECT_FAILURE_COUNTER = 0 # сбрасываем
                        REMEMBERED_W_D_P_SEQUENCE = [] # массив с запомненными вариантами тайм-слотов
                        while fitness_changed_better is False and SUBJECT_FAILURE_COUNTER < max_failure_iterations: # 
                            # попытаемся выбрать рандомно ячейку с парой для дальнейшего swap'а найденной пары:
                            if debug: print("\n\t\t\t>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
                            if debug: print("\t\t\tSUBJECT_FAILURE_COUNTER:", SUBJECT_FAILURE_COUNTER, "/", max_failure_iterations, "|", "fitness_changed_better:", fitness_changed_better)

                            try:
                                
                                prev_draft_fitness = draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS']
                                if debug: print("\t\t\t\tprev_draft_fitness:", prev_draft_fitness)

                                # ---------------------------------------- SWAP ----------------------------------------
                                if debug: print("\t\t\t\t\tv----Попытка поменятся : [weekID =", weekID, ", dayID =", dayID, ", pairID =", pairID, "]")
                                if debug: print("\t\t\t\t\t|", draft_fragment.WEEKS[weekID].visualize(printResult = False, returnResult = True),"\n\t\t\t\t\t| А именно:", draft_fragment.WEEKS[weekID].DAYS[dayID].visualize(printResult = False, returnResult = True))
                                if debug: # с...
                                    print("\t\t\t\t\t>-------------", end="")
                                    for day_gap in range(pairID + 1):
                                        print("--", end="")
                                    print("^")
                                
                                # максимальное количество попыток для нахождения свободной ячейки будет равносильно количеству всех возможных ячеек (помноженные длины массивов каждых сущностей друг на друга)
                                # freeWeekID, freeDayID, freeTimeSlotID = draft_fragment.get_random_free_cell(maximum_attempts = (len(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS) * len(draft_fragment.WEEKS[weekID].DAYS) * len(draft_fragment.WEEKS)) ) # ищем свободную ячейку
                                pairCellWeekID, pairCellDayID, pairCellTimeSlotID = draft_fragment.get_random_pair_cell( maximum_attempts = (len(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS) * len(draft_fragment.WEEKS[weekID].DAYS) * len(draft_fragment.WEEKS)) )

                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t|Попытка поменятся на: [pairCellWeekID =", pairCellWeekID, ", pairCellDayID =", pairCellDayID, ", pairCellTimeSlotID =", pairCellTimeSlotID, "]")
                                if debug: print("\t\t\t\t\t|", draft_fragment.WEEKS[pairCellWeekID].visualize(printResult = False, returnResult = True),"\n\t\t\t\t\t| А именно на:", draft_fragment.WEEKS[pairCellWeekID].DAYS[pairCellDayID].visualize(printResult = False, returnResult = True))
                                if debug: # в...
                                    print("\t\t\t\t\t>------------", end="")
                                    for day_gap in range(pairCellTimeSlotID + 1):
                                        # if day_gap == 0: print("", end="")
                                        # else:
                                            # print("--", end="")
                                            print("--", end="")
                                    for addition in range(day_gap):
                                        print("-", end="")
                                    print("^")

                                
                                if [pairCellWeekID, pairCellDayID, pairCellTimeSlotID] in REMEMBERED_W_D_P_SEQUENCE: # если такая выборка уже происходила
                                    if debug: print("\t\t\t\t\tТакой [pairCellWeekID =", pairCellWeekID, ", pairCellDayID =", pairCellDayID, ", pairCellTimeSlotID =", pairCellTimeSlotID, "] уже рассматривался")
                                    continue # заставляем итерацию while-цикла повторится 


                                # draft_fragment.WEEKS[weekID].DAYS[dayID].move_pair(pairID, pairCellWeekID, pairCellDayID, pairCellTimeSlotID) # перемещаем текущую pairID пару на найденные свободные позиции: pairCellWeekID, pairCellDayID, pairCellTimeSlotID
                                draft_fragment.WEEKS[weekID].DAYS[dayID].swap_pairs(pairID, pairCellWeekID, pairCellDayID, pairCellTimeSlotID) # меняем пары местами
                                
                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t|\t ------------------------------------------------------------------------------------------------------------")
                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t| Результат 'с':", draft_fragment.WEEKS[weekID].visualize(printResult = False, returnResult = True), "\n\t\t\t\t\t| А именно с:", draft_fragment.WEEKS[weekID].DAYS[dayID].visualize(printResult = False, returnResult = True))
                                if debug: # с...
                                    print("\t\t\t\t\t>-------------", end="")
                                    for day_gap in range(pairID + 1):
                                        print("--", end="")
                                    print("^")

                                if debug: print("\t\t\t\t\t| \n\t\t\t\t\t| Результат 'в':", draft_fragment.WEEKS[pairCellWeekID].visualize(printResult = False, returnResult = True), "\n\t\t\t\t\t| А именно в:", draft_fragment.WEEKS[pairCellWeekID].DAYS[pairCellDayID].visualize(printResult = False, returnResult = True))
                                if debug: # в...
                                    print("\t\t\t\t\t>------------", end="")
                                    for day_gap in range(pairCellTimeSlotID + 1):
                                        # if day_gap == 0: print("", end="")
                                        # else:
                                            # print("--", end="")
                                            print("--", end="")
                                    for addition in range(day_gap):
                                        print("-", end="")
                                    print("^")

                                # if debug: print("\t\t\t\t\t", draft_fragment.visualize())
                                # --------------------------------------- /SWAP ----------------------------------------

                                draft_fragment.timeTable.calc_fitness()

                                curr_draft_fitness = draft_fragment.STATISTICS['WHOLE_GENERAL_FITNESS']
                                if debug: print("\t\t\t\tcurr_draft_fitness:", curr_draft_fitness)
                                # if debug: print("\t\t\t\tTEST OF curr_draft_fitness:", isinstance(draft_fragment.WEEKS[weekID].DAYS[dayID], draft_fragment.Week.Day))#draft_fragment.WEEKS[weekID].DAYS[dayID].group.STATISTICS['WHOLE_GENERAL_FITNESS'])

                                # Если удалось улучшить
                                if curr_draft_fitness < prev_draft_fitness:
                                    print("\tImproved fitness: from",prev_draft_fitness, "to", curr_draft_fitness)
                                    fitness_changed_better = True # сообщаем что fitness изменился в лучшую сторону
                                    relevant_fragment = copy.deepcopy(draft_fragment) # актуализируем слепок улучшенным вариантом
                                    SUBJECT_FAILURE_COUNTER = 0 # в любом случае сбрасываем счетчик накопленных ошибочных действий подряд
                                    REMEMBERED_W_D_P_SEQUENCE = [] # запоминаем что такой слот уже посмотрели
                                # Если удалось только лишь ухудшить
                                else:
                                    REMEMBERED_W_D_P_SEQUENCE.append( [pairCellWeekID, pairCellDayID, pairCellTimeSlotID] ) # запоминаем что такой слот уже посмотрели
                                    SUBJECT_FAILURE_COUNTER += 1
                                    # print("\t!!! draft_fragment WAS CHANGED FROM:", draft_fragment, "|", "WHILE draft_fragment.WEEKS[weekID].DAYS[dayID] IS:", draft_fragment.WEEKS[weekID].DAYS[dayID].group)
                                    draft_fragment = copy.deepcopy(relevant_fragment) # возвращаемся на последний релевантный улучшенный вариант
                                    # print("\t!!! draft_fragment WAS CHANGED TO  :", draft_fragment, "|", "WHILE draft_fragment.WEEKS[weekID].DAYS[dayID] IS:", draft_fragment.WEEKS[weekID].DAYS[dayID].group)

                            # если выбрать рандомно свободную ячейку не удалось:
                            except Exception as e:
                                print("\t\t\t\tВыбор ячейки не удался")
                                print(e)
                                return

                        if fitness_changed_better is False and SUBJECT_FAILURE_COUNTER >= max_failure_iterations: # если улучшений так и не произошло, и достигнут максимальный порог фейловых операций
                            WEEK_FAILURE_COUNTER += 1 # то значит найден +1 предмет, который не удалось переместить ни в какую
                            print("WEEK_FAILURE_COUNTER:", WEEK_FAILURE_COUNTER)

                    else: # если текущая рассматриваемая пара (ячейка) вовсе не пара (например, 0, а не ссылка на пару)
                        if debug: print(draft_fragment.WEEKS[weekID].DAYS[dayID].PAIRS[pairID])
                        continue # пропускаем итерацию

                    if debug: print("\t\t_______________________________________________________")
                if debug: print("\t------------------------------------------------------------------------------------------------------------------------------------------------------------------")

        return relevant_fragment

## **Запуск системы и поиск решений**

In [ ]:
chosen_groups_names = [
    "16БИ-2",
    "16ПМИ",
    "16ПИ",
    "16ФМ"
]

### **Генерируем набор начальных решений для выбранных групп**

In [ ]:
all_initial_solutions_for_groups_pack = init_and_get_initial_solutions(
    particular_groups = chosen_groups_names, # укажем для каких групп будет сгенерирован набор из начальных решений
    solutions_amount_per_group = 5, # количество начальных решений в наборе для каждой группы
    visualize_result = True, # визуальная демонстрация каждого решения
    whole_stats = True # визуальная демонстрация статистики каждого решения
)

############################################################ Рассматриваемая группа: 16БИ-2 ############################################################
GROUPS EXTRACTED AND FILLED:
{'16ПИ': {'degree': 'Бакалавриат', 'edu_program': 'Программная инженерия', 'course': '4 курс'}, '16ФМ': {'degree': 'Бакалавриат', 'edu_program': 'Математика', 'course': '4 курс'}, '16БИ-2': {'degree': 'Бакалавриат', 'edu_program': 'Бизнес-информатика', 'course': '4 курс'}, '16ПМИ': {'degree': 'Бакалавриат', 'edu_program': 'Прикладная математика и информатика', 'course': '4 курс'}}
***********************************************************

subjects_arr: {'16ПИ': ['НИС', 'Технологии IоТ', 'Академическое письмо', 'Экономика программной инженерии'], '16ФМ': ['Академическое письмо', 'Компьютерная топология', 'Теория управления', 'Вычислительная математика', 'Теория вероятностей', 'Практикум по компьютерной топологии', 'Общая физика'], '16БИ-2': ['Анализ требований и проектирование информационных систем', 'Мето

### **Найдем самые лучшие по Fitness'у начальные решения у выбранных групп**

In [ ]:
best_initial_solutions_for_groups_pack = {}
for curr_considering_group in chosen_groups_names: # пройдем по всем выбранным группам
    
    print("############################## Рассматриваемая группа:", curr_considering_group, "##############################")
    solutions_fitnesses = []
    for solution in all_initial_solutions_for_groups_pack[curr_considering_group]:
        solutions_fitnesses.append(solution.STATISTICS['WHOLE_GENERAL_FITNESS'])
    
    max_fitness_value_per_solutions = max(solutions_fitnesses)
    min_fitness_value_per_solutions = min(solutions_fitnesses)
    min_fitness_solution_idx = solutions_fitnesses.index(min_fitness_value_per_solutions)
    print("\tMAX fitness:", max_fitness_value_per_solutions, "| MIN fitness:", min_fitness_value_per_solutions, "| MIN fitness solution index:", min_fitness_solution_idx)
    
    min_fitness_solution_link = all_initial_solutions_for_groups_pack[curr_considering_group][min_fitness_solution_idx]

    print("\n\t____________________ Статистика лучшего решения: ____________________")
    for stat in min_fitness_solution_link.STATISTICS:
        hard_constraint_prefix = "\t"
        if stat in min_fitness_solution_link.timeTable.HARD_CONSTRAINTS_NAMES:
            hard_constraint_prefix = "[HARD]\t"
        
        if stat is "WHOLE_GENERAL_FITNESS":
            print("\t_____________________________________________________________________")
            print("\tFitness данного решения:", min_fitness_solution_link.STATISTICS[stat])
        else:
            print("\t" + hard_constraint_prefix, stat, ":", min_fitness_solution_link.STATISTICS[stat])

    best_initial_solutions_for_groups_pack.update({
        curr_considering_group: min_fitness_solution_link
    })
    print("------------------------------------------------------------------------------------------")

best_initial_solutions_for_groups_pack

############################## Рассматриваемая группа: 16БИ-2 ##############################
	MAX fitness: 277 | MIN fitness: 147 | MIN fitness solution index: 3

	____________________ Статистика лучшего решения: ____________________
	[HARD]	 pairs_amount_per_day : [0, 0]
	[HARD]	 pairs_amount_per_week : [0, 0]
	[HARD]	 pairs_in_the_same_corpuses : [0, 0]
	[HARD]	 auditorium_satisfies_pair_type : [0, 0]
	[HARD]	 subject_pairs_amount_per_eduPlan : [0, 0]
		 windows_amount_per_day : [21, 39]
		 preferable_max_pairs_per_day : [8, 8]
		 preferable_min_pairs_per_day : [23, 23]
		 changing_auditoriums_per_day_amount : [50, 25]
		 saturday_pairs_amount : [7, 12]
		 less_preferable_pair_timeslots : [37, 40]
		 WHOLE_GENERAL_FITNESS : 147
	_____________________________________________________________________
	Fitness данного решения: 147
------------------------------------------------------------------------------------------
############################## Рассматриваемая группа: 16ПМИ #######

{'16БИ-2': <__main__.University.ModuleTimetable.Group at 0x7ff91db6e6d0>,
 '16ПИ': <__main__.University.ModuleTimetable.Group at 0x7ff91d285cd0>,
 '16ПМИ': <__main__.University.ModuleTimetable.Group at 0x7ff91f00e590>,
 '16ФМ': <__main__.University.ModuleTimetable.Group at 0x7ff91cd96650>}

In [ ]:
best_initial_solutions_for_groups_pack['16БИ-2'].visualize()
print()
best_initial_solutions_for_groups_pack['16ПМИ'].visualize()
print()
best_initial_solutions_for_groups_pack['16ПИ'].visualize()
print()
best_initial_solutions_for_groups_pack['16ФМ'].visualize()

{'Анализ требований и проектирование информационных систем': 1, 'Методы машинного обучения в информационной безопасности': 2, 'Академическое письмо': 3, 'Имитационное моделирование': 4}
WEEK №1
	 [[0, 0, 1, 0, 0, 0], [0, 0, 0, 2, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 4, 0, 3, 2, 0], [2, 0, 0, 0, 0, 1]]
WEEK №2
	 [[2, 0, 0, 4, 0, 1], [0, 0, 0, 0, 0, 0], [0, 4, 0, 2, 3, 1], [0, 0, 0, 0, 0, 0], [0, 4, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[2, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 2], [0, 4, 0, 0, 0, 2]]
WEEK №4
	 [[0, 0, 2, 0, 0, 2], [0, 0, 0, 0, 0, 0], [0, 0, 4, 0, 0, 0], [0, 0, 0, 0, 1, 3], [0, 0, 4, 0, 0, 4], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 2, 4, 0, 0, 0], [2, 0, 0, 4, 2, 0], [0, 0, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 2, 2, 0, 2], [3, 2, 1, 0, 0, 4], [0, 0, 0, 0, 0, 0], [0, 0, 0, 3, 0, 4], [0, 2, 0, 4, 0, 1], [0, 4, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 3, 0, 1, 1

### **Попытка улучшения расписания для группы 16БИ-2**

#### *Локальный поиск №1*

In [ ]:
group_16BI_2 = best_initial_solutions_for_groups_pack["16БИ-2"] # извлекаем лучшее сгенерированное решение
group_16BI_2_IMPROVED_LS_1 = local_search_v1(group_16BI_2, debug=False) # отправляем данное решение на улучшение

[BEFORE START] relevant_fragment_fitness: 147
WEEK_FAILURE_COUNTER: 0
	Improved fitness: from 147 to 139
	Improved fitness: from 139 to 135
	Improved fitness: from 135 to 134
	Improved fitness: from 134 to 132
	Improved fitness: from 132 to 131
	Improved fitness: from 131 to 127.5
	Improved fitness: from 127.5 to 125
	Improved fitness: from 125 to 121
	Improved fitness: from 121 to 115
	Improved fitness: from 115 to 114
	Improved fitness: from 114 to 106
	Improved fitness: from 106 to 105
	Improved fitness: from 105 to 103
	Improved fitness: from 103 to 101
	Improved fitness: from 101 to 100
	Improved fitness: from 100 to 99
	Improved fitness: from 99 to 98
	Improved fitness: from 98 to 96
	Improved fitness: from 96 to 94
	Improved fitness: from 94 to 93
	Improved fitness: from 93 to 91
	Improved fitness: from 91 to 89
	Improved fitness: from 89 to 88
	Improved fitness: from 88 to 87
	Improved fitness: from 87 to 86
	Improved fitness: from 86 to 85
	Improved fitness: from 85 to 84
	Imp

In [ ]:
group_16BI_2_IMPROVED_LS_1.visualize()

WEEK №1
	 [[0, 0, 0, 1, 1, 0], [0, 2, 2, 0, 0, 0], [0, 2, 4, 0, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 2, 4, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 4, 2, 4, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 4, 2, 4, 0], [0, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 3, 3, 0, 0], [0, 0, 0, 4, 2, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 4, 4, 4, 0], [0, 0, 2, 2, 0, 0], [0, 0, 0, 2, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 4, 2, 2, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 2, 4, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 4, 4, 4, 0], [0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 3, 1, 0, 0, 0], [0, 0, 0, 0,

In [ ]:
group_16BI_2_IMPROVED_LS_1.STATISTICS

{'WHOLE_GENERAL_FITNESS': 21.5,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [43, 21.5],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

#### *Локальный поиск №2*

In [ ]:
group_16BI_2_IMPROVED_LS_2 = local_search_v2(group_16BI_2_IMPROVED_LS_1, max_failure_iterations=400, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16BI_2_IMPROVED_LS_2.visualize()

WEEK №1
	 [[0, 0, 0, 1, 1, 0], [0, 2, 2, 0, 0, 0], [0, 4, 4, 0, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 4, 4, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0], [0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 4, 2, 4, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 4, 4, 4, 0], [0, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 3, 3, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 4, 4, 4, 0], [0, 0, 2, 2, 0, 0], [0, 0, 0, 2, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 2, 2, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 4, 4, 4, 0], [0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 3, 1, 0, 0, 0], [0, 0, 0, 0,

In [ ]:
group_16BI_2_IMPROVED_LS_2.STATISTICS

{'WHOLE_GENERAL_FITNESS': 19.5,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [39, 19.5],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

### **Попытка улучшения расписания для группы 16ПМИ**

#### *Локальный поиск №1*

In [ ]:
group_16PMI = best_initial_solutions_for_groups_pack["16ПМИ"] # извлекаем лучшее сгенерированное решение
group_16PMI_IMPROVED_LS_1 = local_search_v1(group_16PMI, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16PMI_IMPROVED_LS_1.visualize()

WEEK №1
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 0, 0, 0], [0, 0, 6, 2, 2, 0], [0, 0, 2, 2, 0, 0], [0, 4, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 0, 3, 3, 0, 0], [0, 0, 2, 6, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 6, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 2, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 6, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 4, 4, 4, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1,

In [ ]:
group_16PMI_IMPROVED_LS_1.STATISTICS

{'WHOLE_GENERAL_FITNESS': 26.5,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [53, 26.5],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

#### *Локальный поиск №2*

In [ ]:
group_16PMI_IMPROVED_LS_2 = local_search_v2(group_16PMI_IMPROVED_LS_1, max_failure_iterations=400, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16PMI_IMPROVED_LS_2.visualize()

WEEK №1
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 0, 0, 0], [0, 0, 6, 2, 2, 0], [0, 0, 2, 6, 0, 0], [0, 4, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 0, 3, 3, 0, 0], [0, 0, 6, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 3, 3, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 2, 2, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 6, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 4, 4, 4, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 0, 0, 0], [0, 2, 2, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 1, 1,

In [ ]:
group_16PMI_IMPROVED_LS_2.STATISTICS

{'WHOLE_GENERAL_FITNESS': 24,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [48, 24],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

### **Попытка улучшения расписания для группы 16ПИ**

#### *Локальный поиск №1*

In [ ]:
group_16PI = best_initial_solutions_for_groups_pack["16ПИ"] # извлекаем лучшее сгенерированное решение
group_16PI_IMPROVED_LS_1 = local_search_v1(group_16PI, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16PI_IMPROVED_LS_1.visualize()

WEEK №1
	 [[0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 1, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 4, 4, 0], [0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 2, 3, 0, 0, 0], [0, 0, 0, 2, 2, 0], [0, 0, 3, 2, 3, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 2, 0], [0, 2, 3, 2, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 0, 0, 0, 0], [0, 0, 4, 4, 4, 0], [0, 0, 4, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 2, 2, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 0, 0], [0, 0, 1, 1, 0, 0], [0, 4, 4, 4, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 2, 3, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 3, 3, 0], [0, 2, 4, 0,

In [ ]:
group_16PI_IMPROVED_LS_1.STATISTICS

{'WHOLE_GENERAL_FITNESS': 19.5,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [39, 19.5],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

#### *Локальный поиск №2*

In [ ]:
group_16PI_IMPROVED_LS_2 = local_search_v2(group_16PI_IMPROVED_LS_1, max_failure_iterations=400, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16PI_IMPROVED_LS_2.visualize()

WEEK №1
	 [[0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 1, 4, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №2
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 0, 4, 4, 0], [0, 0, 0, 2, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 2, 3, 0, 0, 0], [0, 0, 0, 2, 2, 0], [0, 0, 3, 2, 3, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 2, 2, 0], [0, 2, 3, 2, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 0, 0, 0, 0], [0, 0, 4, 4, 4, 0], [0, 0, 4, 4, 4, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №7
	 [[0, 0, 2, 2, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 2, 2, 0, 0], [0, 0, 1, 1, 0, 0], [0, 4, 4, 4, 0, 0], [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 2, 3, 0, 0, 0], [0, 0, 0, 4, 4, 0], [0, 0, 0, 3, 3, 0], [0, 2, 4, 0,

In [ ]:
group_16PI_IMPROVED_LS_2.STATISTICS

{'WHOLE_GENERAL_FITNESS': 16,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [32, 16],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [0, 0],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

### **Попытка улучшения расписания для группы 16ФМ**

#### *Локальный поиск №1*

In [ ]:
group_16FM = best_initial_solutions_for_groups_pack["16ФМ"] # извлекаем лучшее сгенерированное решение
group_16FM_IMPROVED_LS_1 = local_search_v1(group_16FM, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16FM_IMPROVED_LS_1.visualize()

WEEK №1
	 [[0, 4, 4, 4, 0, 0], [0, 5, 5, 5, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 0, 0, 0, 0], [0, 4, 4, 4, 0, 0],  [0, 3, 6, 7, 0, 0]]
WEEK №2
	 [[0, 0, 0, 1, 5, 0], [0, 1, 1, 2, 0, 0], [0, 0, 1, 2, 0, 0], [0, 0, 7, 7, 3, 0], [0, 2, 6, 5, 0, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 5, 2, 2, 0], [0, 0, 3, 7, 3, 0], [0, 0, 6, 7, 3, 0], [0, 0, 3, 2, 3, 0], [0, 0, 6, 6, 6, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 4, 2, 4, 0, 0], [0, 0, 5, 1, 2, 0], [0, 0, 6, 3, 7, 0], [0, 0, 0, 4, 2, 0], [0, 0, 7, 7, 7, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 5, 6, 5, 0], [0, 0, 1, 2, 1, 0], [0, 0, 0, 0, 0, 0], [0, 6, 6, 3, 0, 0], [0, 4, 4, 6, 0, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 1, 5, 5, 0], [0, 0, 5, 5, 0, 0], [0, 0, 5, 5, 5, 0], [0, 6, 7, 3, 0, 0], [0, 6, 7, 6, 0, 0],  [0, 0, 3, 3, 0, 0]]
WEEK №7
	 [[0, 0, 7, 3, 6, 0], [0, 0, 5, 1, 5, 0], [0, 1, 5, 2, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 2, 2, 2, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 4, 4, 0], [0, 0, 7, 3, 0, 0], [0, 6, 7, 7, 0, 0], [0, 0

In [ ]:
group_16FM_IMPROVED_LS_1.STATISTICS

{'WHOLE_GENERAL_FITNESS': 36,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [58, 29],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [3, 7],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}

#### *Локальный поиск №2*

In [ ]:
group_16FM_IMPROVED_LS_2 = local_search_v2(group_16FM_IMPROVED_LS_1, max_failure_iterations=400, debug=False) # отправляем данное решение на улучшение

In [ ]:
group_16FM_IMPROVED_LS_2.visualize()

WEEK №1
	 [[0, 4, 4, 4, 0, 0], [0, 5, 5, 5, 0, 0], [0, 0, 3, 3, 3, 0], [0, 0, 0, 0, 0, 0], [0, 4, 4, 4, 0, 0],  [0, 3, 7, 7, 0, 0]]
WEEK №2
	 [[0, 0, 0, 1, 5, 0], [0, 1, 1, 2, 0, 0], [0, 0, 1, 2, 0, 0], [0, 0, 7, 7, 3, 0], [0, 2, 6, 5, 0, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №3
	 [[0, 0, 5, 2, 2, 0], [0, 0, 3, 7, 3, 0], [0, 0, 6, 7, 3, 0], [0, 0, 3, 2, 3, 0], [0, 0, 6, 6, 6, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №4
	 [[0, 4, 2, 4, 0, 0], [0, 0, 5, 1, 2, 0], [0, 0, 6, 3, 7, 0], [0, 0, 0, 4, 2, 0], [0, 0, 7, 7, 7, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №5
	 [[0, 0, 5, 6, 5, 0], [0, 0, 1, 2, 1, 0], [0, 0, 0, 0, 0, 0], [0, 6, 6, 6, 0, 0], [0, 4, 4, 6, 0, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №6
	 [[0, 0, 1, 5, 5, 0], [0, 0, 5, 5, 0, 0], [0, 0, 5, 5, 5, 0], [0, 7, 3, 3, 0, 0], [0, 6, 6, 6, 0, 0],  [0, 0, 3, 3, 0, 0]]
WEEK №7
	 [[0, 0, 7, 3, 6, 0], [0, 0, 5, 1, 5, 0], [0, 1, 1, 2, 0, 0], [0, 0, 1, 1, 1, 0], [0, 0, 2, 2, 2, 0],  [0, 0, 0, 0, 0, 0]]
WEEK №8
	 [[0, 0, 0, 4, 4, 0], [0, 0, 3, 3, 0, 0], [0, 6, 7, 7, 0, 0], [0, 0

In [ ]:
group_16FM_IMPROVED_LS_2.STATISTICS

{'WHOLE_GENERAL_FITNESS': 33.5,
 'auditorium_satisfies_pair_type': [0, 0],
 'changing_auditoriums_per_day_amount': [53, 26.5],
 'less_preferable_pair_timeslots': [0, 0],
 'pairs_amount_per_day': [0, 0],
 'pairs_amount_per_week': [0, 0],
 'pairs_in_the_same_corpuses': [0, 0],
 'preferable_max_pairs_per_day': [0, 0],
 'preferable_min_pairs_per_day': [0, 0],
 'saturday_pairs_amount': [3, 7],
 'subject_pairs_amount_per_eduPlan': [0, 0],
 'windows_amount_per_day': [0, 0]}